In [41]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 1: SELETOR DE PASTA COM TIMER + MIGRAÇÃO
# Versão: 4.2 - REVISÃO COM MELHORIAS
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import xlrd
import re
import json
import os
import subprocess
import platform
import shutil
from pathlib import Path
from datetime import datetime
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

import tkinter as tk
from tkinter import filedialog, messagebox, ttk

print("╔" + "="*68 + "╗")
print("║" + " 🔍 PROCESSADOR DE ARQUIVOS DESCONHECIDOS v4.2".center(78) + "║")
print("╠" + "="*68 + "╣")
print("║" + " Timer | Migração | Dicionários | Validações | Logs".center(78) + "║")
print("╚" + "="*68 + "╝")
print("\n✅ Imports carregados")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: LocalizadorDicionario (SISTEMA DE PERSISTÊNCIA GLOBAL)
# ═══════════════════════════════════════════════════════════════════

class LocalizadorDicionario:
    """
    Sistema de localização persistente de dicionários entre sessões.

    Mantém log global em: ~/.processador_dicionario_localizador.json

    Métodos Públicos:
    - obter_dicionario_atual() -> Path  # Para BLOCO 2+
    - obter_pasta_base_atual() -> Path  # Para FileManager
    - registrar_mudanca()               # Chamado por BLOCO 1
    """

    LOG_FILE = Path.home() / '.processador_dicionario_localizador.json'

    @classmethod
    def carregar_log(cls):
        """Carrega log global com fallback para encoding"""
        if cls.LOG_FILE.exists():
            for encoding in ['utf-8', 'utf-8-sig', 'latin-1']:
                try:
                    with open(cls.LOG_FILE, 'r', encoding=encoding) as f:
                        return json.load(f)
                except (UnicodeDecodeError, json.JSONDecodeError):
                    continue
        return {
            'versao': '2.0',
            'dicionario_atual': None,
            'pasta_base_atual': None,
            'historico': []
        }

    @classmethod
    def salvar_log(cls, log):
        """Salva log com backup automático"""
        # Backup do log anterior
        if cls.LOG_FILE.exists():
            backup_file = cls.LOG_FILE.with_suffix('.json.bak')
            shutil.copy2(cls.LOG_FILE, backup_file)

        # Salvar novo log
        with open(cls.LOG_FILE, 'w', encoding='utf-8') as f:
            json.dump(log, f, indent=2, ensure_ascii=False)

    @classmethod
    def obter_dicionario_atual(cls):
        """Retorna Path do dicionário atual (para BLOCO 2+)"""
        log = cls.carregar_log()
        if not log['dicionario_atual']:
            raise FileNotFoundError("❌ Dicionário não encontrado! Execute BLOCO 1.")

        dicionario_path = Path(log['dicionario_atual'])
        if not dicionario_path.exists():
            raise FileNotFoundError(f"❌ Dicionário não existe: {dicionario_path}")

        return dicionario_path

    @classmethod
    def obter_pasta_base_atual(cls):
        """Retorna Path da pasta base atual (para FileManager)"""
        log = cls.carregar_log()
        if not log['pasta_base_atual']:
            raise FileNotFoundError("❌ Pasta base não encontrada! Execute BLOCO 1.")

        pasta_base = Path(log['pasta_base_atual'])
        if not pasta_base.exists():
            raise FileNotFoundError(f"❌ Pasta base não existe: {pasta_base}")

        return pasta_base

    @classmethod
    def registrar_mudanca(cls, pasta_base, dicionario_path, migrado_de=None):
        """Registra mudança de localização no log global"""
        log = cls.carregar_log()

        entrada = {
            'timestamp': datetime.now().isoformat(),
            'pasta_base': str(pasta_base),
            'dicionario_path': str(dicionario_path),
            'existe': dicionario_path.exists()
        }

        if migrado_de:
            entrada['migrado_de'] = str(migrado_de)

        log['dicionario_atual'] = str(dicionario_path)
        log['pasta_base_atual'] = str(pasta_base)
        log['historico'].append(entrada)
        log['ultima_atualizacao'] = datetime.now().isoformat()

        cls.salvar_log(log)

        print(f"\n📍 Localizador atualizado:")
        print(f"   Dicionário: {dicionario_path.name}")
        print(f"   Log: {cls.LOG_FILE}")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: SeletorPastaComTimer (GUI COM TIMER)
# ═══════════════════════════════════════════════════════════════════

class SeletorPastaComTimer:
    """Seletor de pasta destino com timer de 10s e memória"""

    CONFIG_FILE = Path.home() / '.processador_last_directory.json'

    def __init__(self):
        self.resultado = {'path': None, 'acao': None}
        self.timeout_ocorreu = False

    def carregar_ultima_pasta(self):
        """Carrega última pasta usada"""
        if self.CONFIG_FILE.exists():
            try:
                with open(self.CONFIG_FILE, 'r', encoding='utf-8') as f:
                    config = json.load(f)
                ultima_pasta = Path(config.get('last_directory', ''))
                if ultima_pasta.exists():
                    return ultima_pasta
            except:
                pass
        return None

    def salvar_escolha(self, pasta):
        """Salva escolha para próxima execução"""
        config = {
            'last_directory': str(pasta),
            'timestamp': datetime.now().isoformat()
        }
        with open(self.CONFIG_FILE, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2)

    def validar_pasta_destino(self, pasta):
        """Valida se pasta tem permissões adequadas"""
        pasta = Path(pasta)

        # Verificar permissão de escrita
        if not os.access(pasta, os.W_OK):
            return False, "❌ Sem permissão de escrita"

        # Verificar espaço em disco (mínimo 100MB)
        stat = os.statvfs(pasta) if hasattr(os, 'statvfs') else None
        if stat:
            espaco_livre = stat.f_bavail * stat.f_frsize
            if espaco_livre < 100 * 1024 * 1024:  # 100MB
                return False, f"❌ Espaço insuficiente ({espaco_livre/1024/1024:.1f}MB)"

        return True, "✅ Pasta válida"

    def selecionar_com_timer(self):
        """Exibe GUI com timer de 10s"""
        ultima_pasta = self.carregar_ultima_pasta()

        root = tk.Tk()
        root.title("Processador - Pasta Destino")
        root.geometry("650x450")

        # Centralizar janela
        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 225
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        # Título
        tk.Label(frame, text="📂 Pasta DESTINO",
                font=('Arial', 14, 'bold'), bg='white').pack(pady=(0, 15))

        # Mensagem
        if ultima_pasta:
            msg = f"Timer de 10s para usar:\n\n{ultima_pasta}"
        else:
            msg = "Primeira execução - selecione pasta"

        tk.Label(frame, text=msg, justify=tk.LEFT, font=('Arial', 9),
                bg='white', wraplength=600).pack(pady=(0, 15))

        # Timer
        contador = [10]
        if ultima_pasta:
            label_timer = tk.Label(frame, text=f"{contador[0]}s",
                                  font=('Arial', 24, 'bold'),
                                  fg='#FF4444', bg='white')
            label_timer.pack(pady=(5, 20))

            def countdown():
                if contador[0] > 0 and not self.timeout_ocorreu:
                    contador[0] -= 1
                    label_timer.config(text=f"{contador[0]}s")
                    root.after(1000, countdown)
                elif contador[0] == 0:
                    self.timeout_ocorreu = True
                    self.resultado['path'] = ultima_pasta
                    self.resultado['acao'] = 'TIMEOUT'
                    root.quit()
                    root.destroy()

            root.after(1000, countdown)

        # Botões
        def escolher_nova():
            self.timeout_ocorreu = True
            root.withdraw()
            nova_pasta = filedialog.askdirectory(
                title="Pasta DESTINO",
                initialdir=ultima_pasta if ultima_pasta else None
            )

            if nova_pasta:
                valido, msg = self.validar_pasta_destino(nova_pasta)
                if not valido:
                    messagebox.showerror("Pasta Inválida", msg)
                    self.resultado['path'] = ultima_pasta
                    self.resultado['acao'] = 'CANCELADO'
                else:
                    self.resultado['path'] = Path(nova_pasta)
                    self.resultado['acao'] = 'NOVA'
            else:
                self.resultado['path'] = ultima_pasta
                self.resultado['acao'] = 'CANCELADO'

            root.quit()
            root.destroy()

        def usar_ultima():
            self.timeout_ocorreu = True
            self.resultado['path'] = ultima_pasta
            self.resultado['acao'] = 'MANTEVE'
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(frame_btns, text="📁 Nova Pasta", command=escolher_nova,
                 width=20, height=2, font=('Arial', 10, 'bold'),
                 bg='#4CAF50', fg='white').pack(side=tk.LEFT, padx=10)

        if ultima_pasta:
            tk.Button(frame_btns, text="✅ Usar Última", command=usar_ultima,
                     width=20, height=2, font=('Arial', 10),
                     bg='#2196F3', fg='white').pack(side=tk.LEFT, padx=10)

        root.mainloop()
        return self.resultado

# ═══════════════════════════════════════════════════════════════════
# CLASSE: SeletorOrigemComTimer (GUI MIGRAÇÃO)
# ═══════════════════════════════════════════════════════════════════

class SeletorOrigemComTimer:
    """Pergunta se deseja copiar arquivos de execução anterior"""

    def __init__(self):
        self.resultado = {'copiar': False, 'path': None}
        self.timeout_ocorreu = False

    def perguntar_origem(self):
        """GUI com timer de 5s (default: NÃO)"""
        root = tk.Tk()
        root.title("Processador - Copiar Arquivos Anteriores?")
        root.geometry("650x350")

        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 175
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(frame, text="📂 Copiar arquivos de execução anterior?",
                font=('Arial', 14, 'bold'), bg='white').pack(pady=(0, 15))

        tk.Label(frame, text="Se houver dicionários, logs ou outputs anteriores,\nvocê pode copiá-los para a nova estrutura.",
                justify=tk.LEFT, font=('Arial', 9), bg='white',
                wraplength=600).pack(pady=(0, 15))

        contador = [5]
        label_timer = tk.Label(frame, text=f"{contador[0]}s (auto: NÃO)",
                              font=('Arial', 18, 'bold'),
                              fg='#FF6600', bg='white')
        label_timer.pack(pady=(5, 20))

        def countdown():
            if contador[0] > 0 and not self.timeout_ocorreu:
                contador[0] -= 1
                label_timer.config(text=f"{contador[0]}s (auto: NÃO)")
                root.after(1000, countdown)
            elif contador[0] == 0:
                self.timeout_ocorreu = True
                self.resultado['copiar'] = False
                root.quit()
                root.destroy()

        root.after(1000, countdown)

        def sim_copiar():
            self.timeout_ocorreu = True
            root.withdraw()
            pasta_origem = filedialog.askdirectory(
                title="Selecione pasta ORIGEM (execução anterior)"
            )
            if pasta_origem:
                self.resultado['copiar'] = True
                self.resultado['path'] = Path(pasta_origem)
            else:
                self.resultado['copiar'] = False
            root.quit()
            root.destroy()

        def nao_copiar():
            self.timeout_ocorreu = True
            self.resultado['copiar'] = False
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(frame_btns, text="✅ SIM - Selecionar Origem",
                 command=sim_copiar, width=25, height=2,
                 font=('Arial', 10, 'bold'), bg='#4CAF50',
                 fg='white').pack(side=tk.LEFT, padx=10)

        tk.Button(frame_btns, text="❌ NÃO - Começar do Zero",
                 command=nao_copiar, width=25, height=2,
                 font=('Arial', 10), bg='#757575',
                 fg='white').pack(side=tk.LEFT, padx=10)

        root.mainloop()
        return self.resultado

# ═══════════════════════════════════════════════════════════════════
# CLASSE: LimpadorRoot (DETECÇÃO DE POLUIÇÃO)
# ═══════════════════════════════════════════════════════════════════

class LimpadorRoot:
    """Detecta e limpa pastas antigas no root"""

    def __init__(self, pasta_root):
        self.pasta_root = Path(pasta_root)

    def detectar_poluicao(self):
        """Encontra pastas numeradas antigas"""
        pastas_numeradas = [
            p for p in self.pasta_root.iterdir()
            if p.is_dir() and p.name[:2].isdigit() and '_' in p.name
        ]
        return pastas_numeradas

    def perguntar_limpeza(self, pastas):
        """GUI para decidir o que fazer com pastas antigas"""
        root = tk.Tk()
        root.title("Processador - Limpar Root?")
        root.geometry("650x400")

        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 200
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        tk.Label(frame, text="⚠️  Pastas antigas detectadas no root",
                font=('Arial', 14, 'bold'), bg='white',
                fg='#FF6600').pack(pady=(0, 10))

        msg = f"Encontradas {len(pastas)} pastas soltas:\n\n"
        msg += "\n".join([f"• {p.name}" for p in pastas[:5]])
        if len(pastas) > 5:
            msg += f"\n... e mais {len(pastas)-5}"

        tk.Label(frame, text=msg, justify=tk.LEFT, font=('Arial', 9),
                bg='white', wraplength=600).pack(pady=(0, 15))

        resultado = {'acao': None}

        def mover():
            resultado['acao'] = 'MOVER'
            root.quit()
            root.destroy()

        def deletar():
            resultado['acao'] = 'DELETAR'
            root.quit()
            root.destroy()

        def ignorar():
            resultado['acao'] = 'IGNORAR'
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(frame_btns, text="📦 Mover p/ Estrutura", command=mover,
                 width=20, height=2, font=('Arial', 9, 'bold'),
                 bg='#4CAF50', fg='white').pack(side=tk.LEFT, padx=5)

        tk.Button(frame_btns, text="🗑️  Deletar", command=deletar,
                 width=15, height=2, font=('Arial', 9),
                 bg='#F44336', fg='white').pack(side=tk.LEFT, padx=5)

        tk.Button(frame_btns, text="⏭️  Ignorar", command=ignorar,
                 width=15, height=2, font=('Arial', 9),
                 bg='#757575', fg='white').pack(side=tk.LEFT, padx=5)

        root.mainloop()
        return resultado['acao']

# ═══════════════════════════════════════════════════════════════════
# CLASSE: GerenciadorMigracao (CÓPIA COMPLETA COM LOG)
# ═══════════════════════════════════════════════════════════════════

class GerenciadorMigracao:
    """Gerencia cópia completa de execuções anteriores"""

    def __init__(self, pasta_origem, pasta_destino_container):
        self.pasta_origem = Path(pasta_origem)
        self.pasta_destino = Path(pasta_destino_container)
        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        self.log_detalhado = []
        self.erros = []

    def detectar_estrutura(self):
        """Detecta pastas e dicionários na origem"""
        pastas = [
            p for p in self.pasta_origem.iterdir()
            if p.is_dir() and (p.name[:2].isdigit() or 'dicionario' in p.name.lower())
        ]

        dicionarios = []
        pasta_dict = self.pasta_origem / '05_Dicionarios'

        if pasta_dict.exists():
            dicionarios.extend(list(pasta_dict.glob('*.json')))

        dicionarios.extend(list(self.pasta_origem.glob('dicionario*.json')))
        dicionarios = list(set(dicionarios))

        return pastas, dicionarios

    def validar_dicionario(self, arquivo_json):
        """Valida integridade do dicionário JSON"""
        try:
            with open(arquivo_json, 'r', encoding='utf-8') as f:
                data = json.load(f)

            # Verificar estrutura mínima
            if not isinstance(data, dict):
                return False, "JSON não é um dicionário"

            return True, "✅ Válido"
        except json.JSONDecodeError as e:
            return False, f"JSON inválido: {str(e)}"
        except Exception as e:
            return False, f"Erro: {str(e)}"

    def copiar_tudo(self):
        """Copia tudo com tratamento de erros"""
        pastas, dicionarios = self.detectar_estrutura()

        print(f"\n📂 MIGRAÇÃO COMPLETA")
        print("─" * 80)
        print(f"   De: {self.pasta_origem}")
        print(f"   Para: {self.pasta_destino}")
        print(f"   Pastas: {len(pastas)}")
        print(f"   Dicionários: {len(dicionarios)}")
        print()

        if not pastas and not dicionarios:
            print("ℹ️  Nada para copiar")
            return {'migrado': False}

        print("Copiar? (S/N ou Enter=S): ", end='')
        resposta = input().strip().upper()
        if resposta and resposta != 'S':
            print("❌ Migração cancelada")
            return {'migrado': False}

        print(f"\n🔄 Copiando...\n")

        arquivos_copiados = 0
        bytes_copiados = 0
        dicionarios_copiados = []

        # Copiar pastas
        for pasta in sorted(pastas):
            try:
                if pasta.name == '05_Dicionarios':
                    continue

                destino_pasta = self.pasta_destino / pasta.name
                destino_pasta.mkdir(parents=True, exist_ok=True)

                print(f"📁 {pasta.name}", end='')
                arquivos_pasta = 0
                bytes_pasta = 0

                for arquivo in pasta.rglob('*'):
                    if arquivo.is_file():
                        try:
                            destino_arq = destino_pasta / arquivo.relative_to(pasta)
                            destino_arq.parent.mkdir(parents=True, exist_ok=True)
                            shutil.copy2(arquivo, destino_arq)
                            arquivos_copiados += 1
                            arquivos_pasta += 1
                            bytes_pasta += arquivo.stat().st_size
                        except Exception as e:
                            self.erros.append({
                                'arquivo': str(arquivo),
                                'erro': str(e)
                            })

                bytes_copiados += bytes_pasta
                print(f" → {arquivos_pasta} arquivos ({bytes_pasta/1024:.1f} KB)")

                self.log_detalhado.append({
                    'tipo': 'pasta',
                    'nome': pasta.name,
                    'arquivos': arquivos_pasta,
                    'bytes': bytes_pasta
                })

            except Exception as e:
                print(f" ❌ ERRO: {str(e)}")
                self.erros.append({
                    'pasta': pasta.name,
                    'erro': str(e)
                })

        # Copiar dicionários
        if dicionarios:
            pasta_dict_destino = self.pasta_destino / '05_Dicionarios'
            pasta_dict_destino.mkdir(parents=True, exist_ok=True)

            print(f"\n📚 DICIONÁRIOS ({len(dicionarios)}):")

            for dic in dicionarios:
                try:
                    # Validar antes de copiar
                    valido, msg = self.validar_dicionario(dic)

                    destino_dic = pasta_dict_destino / dic.name
                    shutil.copy2(dic, destino_dic)
                    tamanho = dic.stat().st_size

                    status = "✅" if valido else "⚠️"
                    print(f"   {status} {dic.name} ({tamanho/1024:.1f} KB) - {msg}")

                    dicionarios_copiados.append(str(destino_dic))
                    arquivos_copiados += 1
                    bytes_copiados += tamanho

                    self.log_detalhado.append({
                        'tipo': 'dicionario',
                        'nome': dic.name,
                        'bytes': tamanho,
                        'path': str(destino_dic),
                        'validado': valido
                    })

                except Exception as e:
                    print(f"   ❌ {dic.name}: {str(e)}")
                    self.erros.append({
                        'dicionario': dic.name,
                        'erro': str(e)
                    })

        print(f"\n✅ TOTAL: {arquivos_copiados} arquivos, {bytes_copiados/1024/1024:.2f} MB")

        if self.erros:
            print(f"⚠️  {len(self.erros)} erros durante cópia (ver log)")

        # Atualizar localizador se houver dicionários
        if dicionarios_copiados:
            dicionario_principal = max(
                [Path(d) for d in dicionarios_copiados],
                key=lambda p: p.stat().st_mtime
            )
            LocalizadorDicionario.registrar_mudanca(
                pasta_base=self.pasta_destino,
                dicionario_path=dicionario_principal,
                migrado_de=self.pasta_origem
            )

        self._salvar_log_local({
            'migrado': True,
            'arquivos': arquivos_copiados,
            'bytes': bytes_copiados,
            'dicionarios': len(dicionarios_copiados),
            'erros': len(self.erros),
            'detalhes': self.log_detalhado,
            'log_erros': self.erros
        })

        return {
            'migrado': True,
            'arquivos': arquivos_copiados,
            'dicionarios': dicionarios_copiados,
            'erros': self.erros
        }

    def _salvar_log_local(self, info):
        """Salva log detalhado da migração"""
        log_file = self.pasta_destino / 'log_migracoes.json'

        if log_file.exists():
            with open(log_file, 'r', encoding='utf-8') as f:
                historico = json.load(f)
        else:
            historico = {'migracoes': []}

        entrada = {
            'timestamp': self.timestamp,
            'data_hora': datetime.now().isoformat(),
            'pasta_origem': str(self.pasta_origem),
            'pasta_destino': str(self.pasta_destino),
            **info
        }

        historico['migracoes'].append(entrada)
        historico['ultima_migracao'] = self.timestamp

        with open(log_file, 'w', encoding='utf-8') as f:
            json.dump(historico, f, indent=2, ensure_ascii=False)

        print(f"💾 Log: {log_file.name}")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: FileManagerInterativo (GERENCIADOR DE ARQUIVOS)
# ═══════════════════════════════════════════════════════════════════

class FileManagerInterativo:
    """Gerenciador de arquivos e estrutura de pastas"""

    def __init__(self, base_path=None):
        self.base_path = Path(base_path) if base_path else Path.cwd()

        # Estrutura de pastas padrão
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos_integracao': self.base_path / '06_Codigos_Integracao'
        }

        # Criar todas as pastas
        for pasta in self.pastas.values():
            pasta.mkdir(parents=True, exist_ok=True)

        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

        print(f"✅ FileManager inicializado")
        print(f"   📂 Container: {self.base_path.name}")
        print(f"   🕐 Timestamp: {self.timestamp}")

    def salvar(self, df, nome, tipo='xlsx', pasta='processados'):
        """Salva DataFrame na pasta especificada"""
        arquivo = self.pastas[pasta] / f"{nome}_{self.timestamp}.{tipo}"

        if tipo == 'xlsx':
            df.to_excel(arquivo, index=False, engine='openpyxl')
        elif tipo == 'csv':
            df.to_csv(arquivo, index=False, encoding='utf-8-sig')

        return arquivo

    def abrir_pasta(self, pasta):
        """Abre pasta no explorer do sistema"""
        caminho = self.pastas[pasta]
        sistema = platform.system()

        try:
            if sistema == 'Windows':
                os.startfile(caminho)
            elif sistema == 'Darwin':  # macOS
                subprocess.run(['open', caminho])
            else:  # Linux
                subprocess.run(['xdg-open', caminho])
        except Exception as e:
            print(f"⚠️  Não foi possível abrir pasta: {e}")

# ═══════════════════════════════════════════════════════════════════
# FUNÇÃO: gerar_readme
# ═══════════════════════════════════════════════════════════════════

def gerar_readme(pasta_base):
    """Gera README.md na pasta container"""
    readme = f"""# 📚 PROCESSADOR DE ARQUIVOS DESCONHECIDOS

**Gerado:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Pasta:** {pasta_base}

---

## 📁 ESTRUTURA

```
{pasta_base.name}/
├── 01_Entrada/          ← Arquivos originais
├── 02_Processados/      ← Dados limpos
├── 03_Outputs/          ← Resultados finais
├── 04_Logs/             ← Logs de execução
├── 05_Dicionarios/      ← Mapeamentos DE-PARA ⭐
├── 06_Codigos_Integracao/ ← Scripts reutilizáveis
├── README.md            ← Este arquivo
└── log_migracoes.json   ← Histórico de migrações
```

---

## 📚 LOCALIZADOR DE DICIONÁRIO

**Para notebooks consumidores (BLOCO 2+):**

```python
from bloco1 import LocalizadorDicionario

# Obter dicionário atual
dicionario_path = LocalizadorDicionario.obter_dicionario_atual()

# Obter pasta base
pasta_base = LocalizadorDicionario.obter_pasta_base_atual()

# Carregar dicionário
import json
with open(dicionario_path, 'r', encoding='utf-8') as f:
    dicionario = json.load(f)
```

**Log global:** `~/.processador_dicionario_localizador.json`

---

## 🔄 HISTÓRICO DE MIGRAÇÕES

Ver: `log_migracoes.json`

---

## 🆘 SUPORTE

- Erros: `04_Logs/`
- Dicionário perdido: Execute BLOCO 1
- Migração: Consulte `log_migracoes.json`
"""

    readme_path = pasta_base / 'README.md'
    with open(readme_path, 'w', encoding='utf-8') as f:
        f.write(readme)

    print(f"📖 README: {readme_path.name}")

# ═══════════════════════════════════════════════════════════════════
# EXECUÇÃO PRINCIPAL DO BLOCO 1
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔵 ETAPA 1: SELECIONANDO PASTA DESTINO...")
print("="*70)

seletor = SeletorPastaComTimer()
resultado_destino = seletor.selecionar_com_timer()

if not resultado_destino['path']:
    print("❌ Nenhuma pasta selecionada")
    raise ValueError("Execução cancelada")

print(f"\n✅ Destino: {resultado_destino['path']}")
print(f"   Ação: {resultado_destino['acao']}")
seletor.salvar_escolha(resultado_destino['path'])

pasta_root_destino = resultado_destino['path']

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 2: VERIFICANDO POLUIÇÃO NO ROOT...")
print("="*70)

limpador = LimpadorRoot(pasta_root_destino)
pastas_poluidas = limpador.detectar_poluicao()

if pastas_poluidas:
    print(f"\n⚠️  {len(pastas_poluidas)} pastas antigas no root!")
    acao = limpador.perguntar_limpeza(pastas_poluidas)

    if acao == 'DELETAR':
        print("\n🗑️  Deletando...")
        for pasta in pastas_poluidas:
            try:
                shutil.rmtree(pasta)
                print(f"   ✅ {pasta.name}")
            except Exception as e:
                print(f"   ❌ {pasta.name}: {e}")

    elif acao == 'MOVER':
        print("\n📦 Mover será feito após criar container")

    else:
        print("\n⏭️  Ignorando pastas antigas")
else:
    print("✅ Root limpo")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 3: CRIANDO PASTA CONTAINER...")
print("="*70)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
nome_container = f"PROCESSAR_ARQUIVOS_{timestamp}"
pasta_container = pasta_root_destino / nome_container

pasta_container.mkdir(parents=True, exist_ok=True)
print(f"✅ Container: {nome_container}")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 4: COPIAR DE EXECUÇÃO ANTERIOR?")
print("="*70)

seletor_origem = SeletorOrigemComTimer()
resultado_origem = seletor_origem.perguntar_origem()

if resultado_origem['copiar'] and resultado_origem['path']:
    print(f"\n📂 Origem: {resultado_origem['path']}")
    gerenciador_mig = GerenciadorMigracao(resultado_origem['path'], pasta_container)
    info_mig = gerenciador_mig.copiar_tudo()

    if info_mig.get('migrado'):
        print(f"\n✅ Migração concluída")
        if info_mig.get('dicionarios'):
            print(f"   📚 {len(info_mig['dicionarios'])} dicionários copiados")
        if info_mig.get('erros'):
            print(f"   ⚠️  {len(info_mig['erros'])} erros (ver log)")
else:
    print("✅ Começando do zero (sem cópia)")

# Mover pastas antigas se solicitado
if pastas_poluidas and acao == 'MOVER':
    print("\n📦 Movendo pastas antigas para container...")
    for pasta in pastas_poluidas:
        try:
            destino = pasta_container / pasta.name
            if destino.exists():
                shutil.rmtree(destino)
            shutil.move(str(pasta), str(destino))
            print(f"   ✅ {pasta.name}")
        except Exception as e:
            print(f"   ❌ {pasta.name}: {e}")

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🔵 ETAPA 5: INICIALIZANDO FILEMANAGER...")
print("="*70)

fm = FileManagerInterativo(pasta_container)

# Detectar ou registrar dicionário
pasta_dict = fm.pastas['dicionarios']
arquivos_dict = list(pasta_dict.glob('*.json'))

if arquivos_dict:
    dicionario_atual = max(arquivos_dict, key=lambda p: p.stat().st_mtime)
    LocalizadorDicionario.registrar_mudanca(
        pasta_base=pasta_container,
        dicionario_path=dicionario_atual
    )
    print(f"📚 Dicionário detectado: {dicionario_atual.name}")
else:
    dicionario_futuro = pasta_dict / 'dicionario_campos.json'
    LocalizadorDicionario.registrar_mudanca(
        pasta_base=pasta_container,
        dicionario_path=dicionario_futuro
    )
    print(f"ℹ️  Dicionário será criado: {dicionario_futuro.name}")

# Gerar README
gerar_readme(pasta_container)

# ═══════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("✅ BLOCO 1 CONCLUÍDO COM SUCESSO")
print("="*70)
print(f"\n📂 Container: {pasta_container}")
print(f"🕐 Timestamp: {timestamp}")
print(f"📍 Localizador: {LocalizadorDicionario.LOG_FILE}")
print(f"\n📋 Estrutura criada:")
for nome, pasta in fm.pastas.items():
    print(f"   • {pasta.name}")
print("\n" + "="*70)

╔====================================================================╗
║                 🔍 PROCESSADOR DE ARQUIVOS DESCONHECIDOS v4.2                 ║
╠====================================================================╣
║              Timer | Migração | Dicionários | Validações | Logs              ║
╚====================================================================╝

✅ Imports carregados

🔵 ETAPA 1: SELECIONANDO PASTA DESTINO...

✅ Destino: E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)
   Ação: MANTEVE

🔵 ETAPA 2: VERIFICANDO POLUIÇÃO NO ROOT...
✅ Root limpo

🔵 ETAPA 3: CRIANDO PASTA CONTAINER...
✅ Container: PROCESSAR_ARQUIVOS_20251017_120240

🔵 ETAPA 4: COPIAR DE EXECUÇÃO ANTERIOR?
✅ Começando do zero (sem cópia)

🔵 ETAPA 5: INICIALIZANDO FILEMANAGER...
✅ FileManager inicializado
   📂 Container: PROCESSAR_ARQUIVOS_20251017_120240
   🕐 Timestamp: 20251017_120242

📍 Localizador atualizado:
   Dicionário: dicionario_campos.json
   Log: 

In [43]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 2: CLASSES AUXILIARES
# Versão: 4.2 - REVISÃO COM MELHORIAS
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from pathlib import Path
import json
import re
from datetime import datetime
import warnings
from collections import Counter
import os
import platform
import subprocess

warnings.filterwarnings('ignore')

print("\n" + "="*70)
print("📦 BLOCO 2: CLASSES AUXILIARES v4.2")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# CLASSE: LocalizadorDicionario (INTEGRADA DO BLOCO 1)
# ═══════════════════════════════════════════════════════════════════

class LocalizadorDicionario:
    """
    Sistema de localização persistente de dicionários entre sessões.

    Mantém log global em: ~/.processador_dicionario_localizador.json
    """

    LOG_FILE = Path.home() / '.processador_dicionario_localizador.json'

    @classmethod
    def carregar_log(cls):
        """Carrega log global com fallback para encoding"""
        if cls.LOG_FILE.exists():
            for encoding in ['utf-8', 'utf-8-sig', 'latin-1']:
                try:
                    with open(cls.LOG_FILE, 'r', encoding=encoding) as f:
                        return json.load(f)
                except (UnicodeDecodeError, json.JSONDecodeError):
                    continue
        return {
            'versao': '2.0',
            'dicionario_atual': None,
            'pasta_base_atual': None,
            'historico': []
        }

    @classmethod
    def obter_dicionario_atual(cls):
        """Retorna Path do dicionário atual"""
        log = cls.carregar_log()
        if not log['dicionario_atual']:
            raise FileNotFoundError("❌ Dicionário não encontrado! Execute BLOCO 1.")

        dicionario_path = Path(log['dicionario_atual'])
        if not dicionario_path.exists():
            raise FileNotFoundError(f"❌ Dicionário não existe: {dicionario_path}")

        return dicionario_path

    @classmethod
    def obter_pasta_base_atual(cls):
        """Retorna Path da pasta base atual"""
        log = cls.carregar_log()
        if not log['pasta_base_atual']:
            raise FileNotFoundError("❌ Pasta base não encontrada! Execute BLOCO 1.")

        pasta_base = Path(log['pasta_base_atual'])
        if not pasta_base.exists():
            raise FileNotFoundError(f"❌ Pasta base não existe: {pasta_base}")

        return pasta_base

# ═══════════════════════════════════════════════════════════════════
# CLASSE: FileManagerInterativo (INTEGRADA DO BLOCO 1)
# ═══════════════════════════════════════════════════════════════════

class FileManagerInterativo:
    """Gerenciador de arquivos e estrutura de pastas"""

    def __init__(self, base_path):
        self.base_path = Path(base_path)

        # Estrutura de pastas padrão
        self.pastas = {
            'entrada': self.base_path / '01_Entrada',
            'processados': self.base_path / '02_Processados',
            'outputs': self.base_path / '03_Outputs',
            'logs': self.base_path / '04_Logs',
            'dicionarios': self.base_path / '05_Dicionarios',
            'codigos_integracao': self.base_path / '06_Codigos_Integracao'
        }

        # Criar todas as pastas
        for pasta in self.pastas.values():
            pasta.mkdir(parents=True, exist_ok=True)

        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

    def salvar(self, df, nome, tipo='xlsx', pasta='processados'):
        """Salva DataFrame na pasta especificada"""
        arquivo = self.pastas[pasta] / f"{nome}_{self.timestamp}.{tipo}"

        if tipo == 'xlsx':
            df.to_excel(arquivo, index=False, engine='openpyxl')
        elif tipo == 'csv':
            df.to_csv(arquivo, index=False, encoding='utf-8-sig')

        return arquivo

    def abrir_pasta(self, pasta):
        """Abre pasta no explorer do sistema"""
        caminho = self.pastas[pasta]
        sistema = platform.system()

        try:
            if sistema == 'Windows':
                os.startfile(caminho)
            elif sistema == 'Darwin':  # macOS
                subprocess.run(['open', caminho])
            else:  # Linux
                subprocess.run(['xdg-open', caminho])
        except Exception as e:
            print(f"⚠️  Não foi possível abrir pasta: {e}")

# ═══════════════════════════════════════════════════════════════════
# CLASSE: SeletorArquivo (GUI COM TIMER E VALIDAÇÕES)
# ═══════════════════════════════════════════════════════════════════

class SeletorArquivo:
    """Seletor de arquivo com timer de 10s e validações robustas"""

    CONFIG_FILE = Path.home() / '.processador_last_file.json'

    def __init__(self):
        self.resultado = {'path': None, 'acao': None}
        self.timeout_ocorreu = False

    def carregar_ultimo_arquivo(self):
        """Carrega último arquivo usado"""
        if self.CONFIG_FILE.exists():
            try:
                with open(self.CONFIG_FILE, 'r', encoding='utf-8') as f:
                    config = json.load(f)
                ultimo_arquivo = Path(config.get('last_file', ''))
                if ultimo_arquivo.exists():
                    return ultimo_arquivo
            except:
                pass
        return None

    def salvar_escolha(self, arquivo):
        """Salva escolha para próxima execução"""
        config = {
            'last_file': str(arquivo),
            'timestamp': datetime.now().isoformat()
        }
        with open(self.CONFIG_FILE, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2)

    def validar_arquivo(self, arquivo_path):
        """Valida se arquivo é adequado para processamento"""
        arquivo = Path(arquivo_path)

        # Verificar existência
        if not arquivo.exists():
            return False, "❌ Arquivo não existe"

        # Verificar se é arquivo (não diretório)
        if not arquivo.is_file():
            return False, "❌ Não é um arquivo"

        # Verificar permissão de leitura
        if not os.access(arquivo, os.R_OK):
            return False, "❌ Sem permissão de leitura"

        # Verificar tamanho (máximo 500MB)
        tamanho_mb = arquivo.stat().st_size / (1024 * 1024)
        if tamanho_mb > 500:
            return False, f"❌ Arquivo muito grande ({tamanho_mb:.1f}MB > 500MB)"

        # Verificar extensão
        extensoes_validas = {'.xlsx', '.xls', '.csv', '.txt'}
        if arquivo.suffix.lower() not in extensoes_validas:
            return False, f"❌ Extensão inválida ({arquivo.suffix})"

        return True, "✅ Arquivo válido"

    def selecionar_com_timer(self):
        """Exibe GUI com timer de 10s"""
        ultimo_arquivo = self.carregar_ultimo_arquivo()

        root = tk.Tk()
        root.title("Processador - Selecionar Arquivo")
        root.geometry("650x450")

        # Centralizar janela
        x = (root.winfo_screenwidth() // 2) - 325
        y = (root.winfo_screenheight() // 2) - 225
        root.geometry(f"+{x}+{y}")

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        # Título
        tk.Label(frame, text="📁 Selecionar Arquivo",
                font=('Arial', 14, 'bold'), bg='white').pack(pady=(0, 15))

        # Mensagem
        if ultimo_arquivo:
            msg = f"Timer de 10s para usar:\n\n{ultimo_arquivo.name}"
        else:
            msg = "Primeira execução - selecione arquivo"

        tk.Label(frame, text=msg, justify=tk.LEFT, font=('Arial', 9),
                bg='white', wraplength=600).pack(pady=(0, 15))

        # Timer
        contador = [10]
        if ultimo_arquivo:
            label_timer = tk.Label(frame, text=f"{contador[0]}s",
                                  font=('Arial', 24, 'bold'),
                                  fg='#FF4444', bg='white')
            label_timer.pack(pady=(5, 20))

            def countdown():
                if contador[0] > 0 and not self.timeout_ocorreu:
                    contador[0] -= 1
                    label_timer.config(text=f"{contador[0]}s")
                    root.after(1000, countdown)
                elif contador[0] == 0:
                    self.timeout_ocorreu = True
                    self.resultado['path'] = ultimo_arquivo
                    self.resultado['acao'] = 'TIMEOUT'
                    root.quit()
                    root.destroy()

            root.after(1000, countdown)

        # Botões
        def escolher_novo():
            self.timeout_ocorreu = True
            root.withdraw()

            novo_arquivo = filedialog.askopenfilename(
                title="Selecionar Arquivo",
                initialdir=ultimo_arquivo.parent if ultimo_arquivo else None,
                filetypes=[
                    ("Arquivos suportados", "*.xlsx;*.xls;*.csv;*.txt"),
                    ("Excel", "*.xlsx;*.xls"),
                    ("CSV", "*.csv"),
                    ("Todos", "*.*")
                ]
            )

            if novo_arquivo:
                valido, msg = self.validar_arquivo(novo_arquivo)
                if not valido:
                    messagebox.showerror("Arquivo Inválido", msg)
                    self.resultado['path'] = ultimo_arquivo
                    self.resultado['acao'] = 'CANCELADO'
                else:
                    self.resultado['path'] = Path(novo_arquivo)
                    self.resultado['acao'] = 'NOVO'
            else:
                self.resultado['path'] = ultimo_arquivo
                self.resultado['acao'] = 'CANCELADO'

            root.quit()
            root.destroy()

        def usar_ultimo():
            self.timeout_ocorreu = True
            self.resultado['path'] = ultimo_arquivo
            self.resultado['acao'] = 'MANTEVE'
            root.quit()
            root.destroy()

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=15)

        tk.Button(frame_btns, text="📁 Novo Arquivo", command=escolher_novo,
                 width=20, height=2, font=('Arial', 10, 'bold'),
                 bg='#4CAF50', fg='white').pack(side=tk.LEFT, padx=10)

        if ultimo_arquivo:
            tk.Button(frame_btns, text="✅ Usar Último", command=usar_ultimo,
                     width=20, height=2, font=('Arial', 10),
                     bg='#2196F3', fg='white').pack(side=tk.LEFT, padx=10)

        root.mainloop()
        return self.resultado

# ═══════════════════════════════════════════════════════════════════
# CLASSE: DetectorCabecalho (ANÁLISE INTELIGENTE COM LOG)
# ═══════════════════════════════════════════════════════════════════

class DetectorCabecalho:
    """
    Detecta automaticamente a linha de cabeçalho em arquivos.

    Usa sistema de scoring baseado em:
    - Preenchimento (70%+ colunas com dados)
    - Tipo String (80%+ colunas texto)
    - Valores únicos (indicador de rótulos)
    - Palavras-chave típicas de cabeçalho
    - Posição na planilha (primeiras linhas têm prioridade)
    """

    def __init__(self, df):
        self.df = df
        self.scores = []
        self.log_decisoes = []

    def detectar(self, n_linhas=50):
        """
        Analisa primeiras n linhas e retorna índice do cabeçalho.

        Args:
            n_linhas: Número de linhas a analisar

        Returns:
            dict: {
                'indice': int,  # Linha detectada como cabeçalho
                'score': float,  # Confiança da detecção (0-1)
                'metodo': str,   # Como foi detectado
                'scores_todas_linhas': list,  # Para debug
                'log_decisoes': list  # Histórico de análise
            }
        """
        n_linhas = min(n_linhas, len(self.df))
        palavras_chave = [
            'codigo', 'nome', 'descri', 'data', 'valor', 'quantidade',
            'centro', 'produto', 'material', 'sigla', 'tipo', 'grupo'
        ]

        for i in range(n_linhas):
            linha = self.df.iloc[i]
            score = 0
            detalhes = {'linha': i, 'criterios': {}}

            # Critério 1: Preenchimento (30 pontos)
            preenchimento = linha.notna().sum() / len(linha)
            if preenchimento >= 0.7:
                pontos = 30 * (preenchimento - 0.7) / 0.3
                score += pontos
                detalhes['criterios']['preenchimento'] = f"{preenchimento:.1%} (+{pontos:.1f})"

            # Critério 2: Tipo String (30 pontos)
            tipos_string = sum(isinstance(v, str) for v in linha)
            proporcao_string = tipos_string / len(linha)
            if proporcao_string >= 0.8:
                pontos = 30 * (proporcao_string - 0.8) / 0.2
                score += pontos
                detalhes['criterios']['strings'] = f"{proporcao_string:.1%} (+{pontos:.1f})"

            # Critério 3: Valores únicos (20 pontos)
            valores_unicos = len(set(str(v) for v in linha if pd.notna(v)))
            if valores_unicos >= len(linha) * 0.8:
                pontos = 20
                score += pontos
                detalhes['criterios']['unicos'] = f"{valores_unicos}/{len(linha)} (+{pontos})"

            # Critério 4: Palavras-chave (10 pontos)
            texto_linha = ' '.join(str(v).lower() for v in linha if pd.notna(v))
            palavras_encontradas = sum(1 for p in palavras_chave if p in texto_linha)
            if palavras_encontradas > 0:
                pontos = min(10, palavras_encontradas * 3)
                score += pontos
                detalhes['criterios']['palavras'] = f"{palavras_encontradas} palavras (+{pontos})"

            # Critério 5: Posição (10 pontos) - primeiras linhas têm vantagem
            if i < 50:
                pontos = 10 * (1 - (i / 50))
                score += pontos
                detalhes['criterios']['posicao'] = f"linha {i} (+{pontos:.1f})"

            detalhes['score_total'] = score
            self.scores.append(score)
            self.log_decisoes.append(detalhes)

        # Encontrar melhor score
        melhor_indice = self.scores.index(max(self.scores))
        melhor_score = self.scores[melhor_indice]

        # Normalizar score para 0-1
        score_normalizado = min(1.0, melhor_score / 100)

        resultado = {
            'indice': melhor_indice,
            'score': score_normalizado,
            'metodo': 'SCORING_AUTOMATICO',
            'scores_todas_linhas': self.scores,
            'log_decisoes': self.log_decisoes
        }

        return resultado

# ═══════════════════════════════════════════════════════════════════
# INICIALIZAÇÃO DO FILEMANAGER (CONECTANDO COM BLOCO 1)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔧 INICIALIZANDO FILEMANAGER - CONECTANDO COM BLOCO 1")
print("="*70)

try:
    pasta_base = LocalizadorDicionario.obter_pasta_base_atual()

    print(f"✅ Container do BLOCO 1 encontrado!")
    print(f"   📂 {pasta_base}")

    fm = FileManagerInterativo(pasta_base)

except FileNotFoundError as e:
    print(f"\n{e}")
    print("\n⚠️  ATENÇÃO: Execute o BLOCO 1 primeiro!")
    raise

print("\n" + "="*70)
print("✅ BLOCO 2 CONCLUÍDO")
print("="*70)
print("\n📋 Classes carregadas:")
print("   • LocalizadorDicionario ........... ✅")
print("   • FileManagerInterativo ........... ✅")
print("   • SeletorArquivo .................. ✅")
print("   • DetectorCabecalho ............... ✅")
print("\n💾 FileManager ativo:")
print(f"   Base: {fm.base_path}")
print(f"   Timestamp: {fm.timestamp}")
print("\n📍 Estrutura de pastas:")
for nome, pasta in fm.pastas.items():
    print(f"   • {nome.ljust(20)}: {pasta.name}")
print("\n" + "="*70)
print("Digite 'BLOCO 2 OK' para prosseguir ao BLOCO 3")
print("="*70)


📦 BLOCO 2: CLASSES AUXILIARES v4.2

🔧 INICIALIZANDO FILEMANAGER - CONECTANDO COM BLOCO 1
✅ Container do BLOCO 1 encontrado!
   📂 E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)\PROCESSAR_ARQUIVOS_20251017_120240

✅ BLOCO 2 CONCLUÍDO

📋 Classes carregadas:
   • LocalizadorDicionario ........... ✅
   • FileManagerInterativo ........... ✅
   • SeletorArquivo .................. ✅
   • DetectorCabecalho ............... ✅

💾 FileManager ativo:
   Base: E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\_DataLake\2- Dados Processados (PROCESSED)\PROCESSAR_ARQUIVOS_20251017_120240
   Timestamp: 20251017_141810

📍 Estrutura de pastas:
   • entrada             : 01_Entrada
   • processados         : 02_Processados
   • outputs             : 03_Outputs
   • logs                : 04_Logs
   • dicionarios         : 05_Dicionarios
   • codigos_integracao  : 06_Codigos_Integracao

Digite 'BLOCO 2 OK' para prosseguir ao BLOCO 3


In [44]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 3: DICIONÁRIO INTELIGENTE + SELEÇÃO DE ARQUIVO DE DADOS
# Versão: v3.1 - Integração completa + Mensagens claras
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
import json
import re
from pathlib import Path
from datetime import datetime
import tkinter as tk
from tkinter import messagebox

print("\n" + "="*70)
print("📚 BLOCO 3: DICIONÁRIO INTELIGENTE + SELEÇÃO DE ARQUIVO")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# PARTE 1: DICIONÁRIO INTELIGENTE (COMPLETO - 448 LINHAS)
# ═══════════════════════════════════════════════════════════════════

class DicionarioInteligente:
    """Dicionário com detecção avançada e validação de ambiguidades"""

    def __init__(self, fm):
        self.fm = fm
        self.arquivo_dict = fm.pastas['dicionarios'] / 'DICIONARIO_MASTER.json'
        self.dados = self._carregar_ou_criar()

    def _carregar_ou_criar(self):
        if self.arquivo_dict.exists():
            try:
                with open(self.arquivo_dict, 'r', encoding='utf-8') as f:
                    dados = json.load(f)

                if 'campos_conhecidos' not in dados:
                    print(f"\n⚠️  Formato antigo - migrando...")
                    dados = self._migrar_formato_antigo(dados)
                    self._salvar(dados)
                    print(f"✅ Migração concluída")

                n_campos = len(dados['campos_conhecidos'])
                n_arquivos = len(dados.get('historico_arquivos', []))

                print(f"\n✅ DICIONÁRIO MASTER CARREGADO")
                print(f"   📚 {n_campos} campos conhecidos")
                print(f"   📁 {n_arquivos} arquivos processados")

                return dados

            except Exception as e:
                print(f"\n⚠️  Erro ao carregar: {e}")
                print(f"   Criando novo dicionário...")
                dados = self._criar_novo()
                self._salvar(dados)
                return dados
        else:
            print(f"\n📝 CRIANDO NOVO DICIONÁRIO MASTER...")
            dados = self._criar_novo()
            self._salvar(dados)
            print(f"✅ Dicionário criado: {len(dados['campos_conhecidos'])} campos padrão")
            return dados

    def _migrar_formato_antigo(self, dados_antigos):
        novo = self._criar_novo()
        if 'arquivos_processados' in dados_antigos:
            novo['historico_arquivos'] = dados_antigos['arquivos_processados']
        return novo

    def _criar_novo(self):
        """Dicionário completo expandido"""
        return {
            'versao': '3.1',
            'criado_em': datetime.now().isoformat(),
            'ultima_atualizacao': datetime.now().isoformat(),

            'campos_conhecidos': {
                'Centro': {
                    'tipo_dado': 'Codigo_Centro',
                    'regex': r'^[5-9]\d{3}$',
                    'sinonimos': ['Centro', 'Código de Centro', 'Cod Centro', 'Unidade Operacional:Centro'],
                    'exemplos': ['5025', '5065', '5174'],
                    'descricao': 'Código numérico de 4 dígitos',
                    'nao_confundir_com': ['Sigla'],
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Sigla': {
                    'tipo_dado': 'Sigla_Base',
                    'regex': r'^[A-Z]{4,10}$',
                    'sinonimos': ['Sigla', 'Sigla Base', 'Sigla Centro', 'Base'],
                    'exemplos': ['BABET', 'BAPLAN', 'AIBET'],
                    'descricao': 'Sigla alfabética (4-10 letras maiúsculas)',
                    'nao_confundir_com': ['Centro'],
                    'relacionamento': 'Um Centro pode ter múltiplas Siglas (filiais)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Codigo_Produto': {
                    'tipo_dado': 'Codigo_Material',
                    'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$',
                    'sinonimos': ['Código Produto', 'Código Material', 'Cod Produto', 'Cod Material'],
                    'exemplos': ['10.123.456', '1.234.567', '1234567'],
                    'descricao': 'Código numérico do material/produto (formato: xx.xxx.xxx)',
                    'nao_confundir_com': ['Codigo_Grupo_Produto'],
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Codigo_Grupo_Produto': {
                    'tipo_dado': 'Codigo_Grupo',
                    'regex': r'^\d{1,2}\.\d{3}\.\d{3}$|^\d{7,8}$|^[A-Z_]+$',
                    'sinonimos': ['Cód Grupo de produto', 'Código Grupo', 'Grupo Produto', 'Produto:CodGrupoProduto'],
                    'exemplos': ['10.123.456', 'DIESEL_S10_SIMPLES', 'GASOLINA_COMUM'],
                    'descricao': 'Código do grupo de produto (numérico OU texto com underscore)',
                    'nao_confundir_com': ['Codigo_Produto'],
                    'diferenca_chave': 'Aceita TEXTO_COM_UNDERSCORE, Codigo_Produto não',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Desc_Grupo_Produto': {
                    'tipo_dado': 'Texto_Descricao',
                    'regex': r'^[A-Za-z0-9\s\-]+$',
                    'sinonimos': ['Desc. Grupo de Produto', 'Descrição Produto', 'Nome Produto', 'Produto'],
                    'exemplos': ['DIESEL S10', 'GASOLINA COMUM', 'ETANOL HIDRATADO'],
                    'descricao': 'Descrição textual do grupo de produto',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Nome_Pessoa': {
                    'tipo_dado': 'Texto_Nome_Pessoa',
                    'regex': r'^[A-ZÁÀÂÃÉÈÊÍÏÓÔÕÖÚÇÑ][a-záàâãéèêíïóôõöúçñ]+(\s[A-ZÁÀÂÃÉÈÊÍÏÓÔÕÖÚÇÑ][a-záàâãéèêíïóôõöúçñ]+)+$',
                    'sinonimos': ['Criado por', 'Nome', 'Responsável', 'Solicitante'],
                    'exemplos': ['Kenedy Vinícius Rodrigues', 'Joao Carlos Stival', 'Lucinéia Lopes de Barros'],
                    'descricao': 'Nome completo de pessoa',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Email': {
                    'tipo_dado': 'Texto_Email',
                    'regex': r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$',
                    'sinonimos': ['Email', 'E-mail', 'Modificado por'],
                    'exemplos': ['usuario@vibraenergia.com.br', 'kenedyvr@vibraenergia.com.br'],
                    'descricao': 'Endereço de email',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Status_Workflow': {
                    'tipo_dado': 'Texto_Status',
                    'regex': r'^(Em aprovação|Aprovado|Rejeitado|Pendente|Concluído|Ítem Criado|Item Criado)$',
                    'sinonimos': ['Status', 'Status Aprovação', 'Situação'],
                    'exemplos': ['Em aprovação', 'Ítem Criado', 'Aprovado'],
                    'descricao': 'Status de workflow/aprovação',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Booleano_Texto': {
                    'tipo_dado': 'Texto_Booleano',
                    'regex': r'^(Sim|Não|sim|não|SIM|NÃO|Yes|No|TRUE|FALSE)$',
                    'sinonimos': ['Concluído?', 'Ativo?', 'Habilitado?'],
                    'exemplos': ['Sim', 'Não'],
                    'descricao': 'Valor booleano como texto',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Texto_Longo': {
                    'tipo_dado': 'Texto_Justificativa',
                    'regex': r'^.{50,}$',
                    'sinonimos': ['Justificativa', 'Observação', 'Comentário', 'Descrição Detalhada'],
                    'exemplos': ['Solicitamos a revisão do limite técnico AIVI devido ao...'],
                    'descricao': 'Texto longo (mais de 50 caracteres)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Path_URL': {
                    'tipo_dado': 'Texto_Caminho',
                    'regex': r'^(teams/|http|https|ftp|\\\\|/).*',
                    'sinonimos': ['Caminho', 'Path', 'URL', 'Link'],
                    'exemplos': ['teams/portaleso/Lists/Solicitao...', 'https://example.com'],
                    'descricao': 'Caminho de arquivo ou URL',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Sigla_Curta': {
                    'tipo_dado': 'Texto_Sigla_Curta',
                    'regex': r'^[A-Z]{2,4}$',
                    'sinonimos': ['CME', 'Gerência', 'UF', 'Tipo'],
                    'exemplos': ['OPC', 'OPN', 'Norte', 'Sul', 'CME'],
                    'descricao': 'Sigla curta (2-4 letras maiúsculas)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Tipo_Item': {
                    'tipo_dado': 'Texto_Tipo_Item',
                    'regex': r'^(Item|Documento|Pasta|Arquivo)$',
                    'sinonimos': ['Tipo de Item', 'Tipo'],
                    'exemplos': ['Item'],
                    'descricao': 'Tipo de item em lista SharePoint',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Data_ISO': {
                    'tipo_dado': 'Data_YYYY-MM-DD',
                    'regex': r'^\d{4}-\d{2}-\d{2}$',
                    'sinonimos': ['Data', 'Período', 'Período Início Validade Novo Limite'],
                    'exemplos': ['2025-08-01', '2024-12-31', '2025-01-07'],
                    'descricao': 'Data formato ISO (YYYY-MM-DD)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Datetime_ISO': {
                    'tipo_dado': 'Datetime_YYYY-MM-DD_HH:MM:SS',
                    'regex': r'^\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}$',
                    'sinonimos': ['Criado', 'Modificado', 'Data Hora', 'Timestamp'],
                    'exemplos': ['2025-08-04 19:22:17', '2025-08-04 20:45:37'],
                    'descricao': 'Data e hora formato ISO',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Data_BR': {
                    'tipo_dado': 'Data_DD/MM/YYYY',
                    'regex': r'^\d{2}/\d{2}/\d{4}$',
                    'sinonimos': ['Data'],
                    'exemplos': ['15/01/2024', '31/12/2025'],
                    'descricao': 'Data formato brasileiro',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Percentual_Decimal': {
                    'tipo_dado': 'Numero_Percentual_Decimal',
                    'regex': r'^-?\d+(\.\d+)?$',
                    'sinonimos': ['Limite Inferior Atual', 'Limite Superior Atual', 'AVG VI %', '% VI', 'AVG VI % 2024 SAP'],
                    'exemplos': ['-0.08', '0.08', '-0.14', '0.05', '-0.03'],
                    'descricao': 'Percentual em formato decimal (pode ser negativo)',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Percentual_Com_Simbolo': {
                    'tipo_dado': 'Numero_Percentual_Simbolo',
                    'regex': r'^-?\d+(\.\d+)?%$',
                    'sinonimos': ['Percentual', '% VI'],
                    'exemplos': ['10.5%', '-5.2%', '100%'],
                    'descricao': 'Percentual com símbolo %',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Numero_Inteiro': {
                    'tipo_dado': 'Numero_Inteiro',
                    'regex': r'^-?\d+$',
                    'sinonimos': ['Quantidade', 'Qtd', 'Total'],
                    'exemplos': ['123', '-456', '1000'],
                    'descricao': 'Número inteiro',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Monetario': {
                    'tipo_dado': 'Numero_Monetario',
                    'regex': r'^R\$\s?-?\d{1,3}(\.\d{3})*(,\d{2})?$|^-?\d+([.,]\d{2})?$',
                    'sinonimos': ['Valor', 'Custo', 'Preço', 'R$'],
                    'exemplos': ['R$ 1.234,56', '1234.56'],
                    'descricao': 'Valor monetário',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Unidade_Operacional_Nome': {
                    'tipo_dado': 'Texto_Unidade_Operacional',
                    'regex': r'^[A-Z]{4,10}\s+Base\s+de\s+.+$',
                    'sinonimos': ['Unidade Operacional', 'Nome Base'],
                    'exemplos': ['BABET Base de Betim', 'BAPLAN Base de Paulínia', 'BADEN Base de Presid Prudente'],
                    'descricao': 'Nome completo da unidade operacional',
                    'aprendido_de': 'PADRAO_INICIAL'
                },
                'Rotulo_Retencao': {
                    'tipo_dado': 'Texto_Rotulo_Vazio',
                    'regex': r'^(NaN|nan|None|null|)$',
                    'sinonimos': ['Rótulo de retenção Aplicado'],
                    'exemplos': ['NaN'],
                    'descricao': 'Campo de rótulo (geralmente vazio)',
                    'aprendido_de': 'PADRAO_INICIAL'
                }
            },

            'historico_arquivos': []
        }

    def detectar_campo(self, coluna_nome, valores_amostra):
        """Detecção AVANÇADA com múltiplas estratégias"""
        valores_str = [str(v).strip() for v in valores_amostra if pd.notna(v) and str(v).strip() not in ['', 'nan', 'None']]

        if not valores_str:
            return {
                'campo_detectado': 'VAZIO',
                'confianca': 0.0,
                'score_conteudo': 0.0,
                'score_nome': 0.0,
                'matches': 0,
                'total': 0,
                'ambiguidade': False,
                'candidatos': [],
                'metodo': 'VAZIO'
            }

        # ESTRATÉGIA 1: Heurísticas específicas
        campo_heuristico = self._detectar_por_heuristica(coluna_nome, valores_str)

        if campo_heuristico and campo_heuristico.get('confianca', 0) >= 0.85:
            campo_heuristico.setdefault('score_nome', 0.0)
            campo_heuristico.setdefault('score_conteudo', campo_heuristico.get('confianca', 0.0))
            campo_heuristico.setdefault('matches', len(valores_str))
            campo_heuristico.setdefault('total', len(valores_str))
            campo_heuristico.setdefault('ambiguidade', False)
            campo_heuristico.setdefault('candidatos', [])
            return campo_heuristico

        # ESTRATÉGIA 2: Match por regex
        resultados_regex = []
        for nome_campo, info in self.dados['campos_conhecidos'].items():
            matches = sum(1 for v in valores_str if re.match(info['regex'], v))
            score_conteudo = matches / len(valores_str)

            score_nome = 0.0
            for sinonimo in info['sinonimos']:
                if sinonimo.lower() in coluna_nome.lower():
                    score_nome = 0.3
                    break

            score_final = score_conteudo + score_nome

            resultados_regex.append({
                'campo': nome_campo,
                'score': min(score_final, 1.0),
                'score_conteudo': score_conteudo,
                'score_nome': score_nome,
                'matches': matches,
                'total': len(valores_str)
            })

        resultados_regex = sorted(resultados_regex, key=lambda x: x['score'], reverse=True)
        melhor_regex = resultados_regex[0]
        segundo_regex = resultados_regex[1] if len(resultados_regex) > 1 else None

        ambiguidade = False
        candidatos = []
        if segundo_regex and abs(melhor_regex['score'] - segundo_regex['score']) < 0.10:
            ambiguidade = True
            candidatos = [segundo_regex['campo']]

        resultado_final = {
            'campo_detectado': melhor_regex['campo'],
            'confianca': melhor_regex['score'],
            'score_conteudo': melhor_regex['score_conteudo'],
            'score_nome': melhor_regex['score_nome'],
            'matches': melhor_regex['matches'],
            'total': melhor_regex['total'],
            'ambiguidade': ambiguidade,
            'candidatos': candidatos,
            'metodo': 'REGEX'
        }

        if resultado_final['confianca'] < 0.50:
            resultado_final['campo_detectado'] = 'DESCONHECIDO'

        return resultado_final

    def _detectar_por_heuristica(self, nome_coluna, valores_str):
        """Detecção por heurísticas específicas"""
        if not valores_str:
            return None

        tamanho_medio = sum(len(v) for v in valores_str) / len(valores_str)
        valores_unicos = set(valores_str)

        # HEURÍSTICAS COMPLETAS
        if all(re.match(r'^\d{4}-\d{2}-\d{2}$', v) for v in valores_str[:5]):
            return {'campo_detectado': 'Data_ISO', 'confianca': 0.95, 'metodo': 'HEURISTICA_DATA_ISO'}

        if all(re.match(r'^\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}', v) for v in valores_str[:5]):
            return {'campo_detectado': 'Datetime_ISO', 'confianca': 0.95, 'metodo': 'HEURISTICA_DATETIME'}

        if all('@' in v for v in valores_str):
            return {'campo_detectado': 'Email', 'confianca': 0.95, 'metodo': 'HEURISTICA_EMAIL'}

        if 'limite' in nome_coluna.lower() or '%' in nome_coluna or 'vi' in nome_coluna.lower():
            try:
                valores_float = [float(v) for v in valores_str if v not in ['nan', '', 'None', 'NaN']]
                if valores_float and all(-1 <= v <= 1 for v in valores_float):
                    return {'campo_detectado': 'Percentual_Decimal', 'confianca': 0.90, 'metodo': 'HEURISTICA_PERCENTUAL'}
            except:
                pass

        if tamanho_medio > 50:
            return {'campo_detectado': 'Texto_Longo', 'confianca': 0.85, 'metodo': 'HEURISTICA_TEXTO_LONGO'}

        if any(v.startswith(('teams/', 'http', 'https', '//', '\\\\')) for v in valores_str):
            return {'campo_detectado': 'Path_URL', 'confianca': 0.90, 'metodo': 'HEURISTICA_PATH'}

        if valores_unicos <= {'Sim', 'Não', 'sim', 'não'}:
            return {'campo_detectado': 'Booleano_Texto', 'confianca': 0.95, 'metodo': 'HEURISTICA_BOOLEANO'}

        if 'por' in nome_coluna.lower() or 'nome' in nome_coluna.lower():
            if all(len(v.split()) >= 2 and '@' not in v for v in valores_str[:5]):
                return {'campo_detectado': 'Nome_Pessoa', 'confianca': 0.85, 'metodo': 'HEURISTICA_NOME'}

        palavras_status = {'em aprovação', 'aprovado', 'rejeitado', 'ítem criado', 'item criado', 'pendente'}
        if any(v.lower() in palavras_status for v in valores_str):
            return {'campo_detectado': 'Status_Workflow', 'confianca': 0.90, 'metodo': 'HEURISTICA_STATUS'}

        if 'base de' in ' '.join(valores_str[:3]).lower():
            return {'campo_detectado': 'Unidade_Operacional_Nome', 'confianca': 0.90, 'metodo': 'HEURISTICA_UNIDADE_OP'}

        return None

    def atualizar_historico(self, info):
        self.dados['historico_arquivos'].append(info)
        self.dados['ultima_atualizacao'] = datetime.now().isoformat()
        self._salvar(self.dados)

    def _salvar(self, dados):
        with open(self.arquivo_dict, 'w', encoding='utf-8') as f:
            json.dump(dados, f, indent=2, ensure_ascii=False)

# ═══════════════════════════════════════════════════════════════════
# PARTE 2: INICIALIZAR DICIONÁRIO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📚 INICIALIZANDO DICIONÁRIO INTELIGENTE")
print("="*70)
print("\nℹ️  O dicionário armazena PADRÕES de campos conhecidos")
print("   para ajudar a identificar colunas automaticamente.")

dicionario = DicionarioInteligente(fm)

# ═══════════════════════════════════════════════════════════════════
# PARTE 3: SELEÇÃO DE ARQUIVO DE DADOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📂 SELEÇÃO DE ARQUIVO DE DADOS")
print("="*70)
print("\n⚠️  ATENÇÃO: Selecione o arquivo DE DADOS a processar")
print("   (Excel, CSV ou TXT com os dados)")
print("   NÃO selecione o arquivo do dicionário!")

seletor = SeletorArquivo()
resultado = seletor.selecionar_com_timer()

if not resultado['path']:
    print("\n❌ Nenhum arquivo selecionado")
    raise ValueError("Execução cancelada pelo usuário")

arquivo_selecionado = resultado['path']
acao = resultado['acao']

# Validação de extensão
extensao = arquivo_selecionado.suffix.lower()
if extensao not in ['.xlsx', '.xls', '.csv', '.txt']:
    print(f"\n⚠️  AVISO: Extensão '{extensao}' não é comum")
    print("   Esperado: .xlsx, .xls, .csv ou .txt")

print(f"\n✅ ARQUIVO DE DADOS SELECIONADO:")
print(f"   📄 Nome: {arquivo_selecionado.name}")
print(f"   📁 Pasta: {arquivo_selecionado.parent.name}")
print(f"   📊 Tipo: {extensao.upper()}")
print(f"   🎯 Ação: {acao}")
print(f"   💾 Tamanho: {arquivo_selecionado.stat().st_size / 1024:.1f} KB")

# Salvar escolha
seletor.salvar_escolha(arquivo_selecionado)

print("\n" + "="*70)
print("✅ BLOCO 3 CONCLUÍDO")
print("="*70)
print("\n📋 Componentes ativos:")
print("   • DicionarioInteligente ........... ✅")
print(f"     - Campos conhecidos: {len(dicionario.dados['campos_conhecidos'])}")
print(f"     - Arquivos processados: {len(dicionario.dados['historico_arquivos'])}")
print("   • SeletorArquivo .................. ✅")
print(f"     - Arquivo de dados: {arquivo_selecionado.name}")
print(f"     - Tipo: {extensao.upper()}")
print("\n💡 Próximo passo:")
print("   BLOCO 4 vai detectar o cabeçalho e carregar os dados")
print("\n" + "="*70)
print("Digite 'BLOCO 3 OK' para prosseguir ao BLOCO 4")
print("="*70)


📚 BLOCO 3: DICIONÁRIO INTELIGENTE + SELEÇÃO DE ARQUIVO

📚 INICIALIZANDO DICIONÁRIO INTELIGENTE

ℹ️  O dicionário armazena PADRÕES de campos conhecidos
   para ajudar a identificar colunas automaticamente.

📝 CRIANDO NOVO DICIONÁRIO MASTER...
✅ Dicionário criado: 22 campos padrão

📂 SELEÇÃO DE ARQUIVO DE DADOS

⚠️  ATENÇÃO: Selecione o arquivo DE DADOS a processar
   (Excel, CSV ou TXT com os dados)
   NÃO selecione o arquivo do dicionário!

✅ ARQUIVO DE DADOS SELECIONADO:
   📄 Nome: Cópia de xSAPtemp4687_JAN_25.xls
   📁 Pasta: Dado BW
   📊 Tipo: .XLS
   🎯 Ação: MANTEVE
   💾 Tamanho: 16257.5 KB

✅ BLOCO 3 CONCLUÍDO

📋 Componentes ativos:
   • DicionarioInteligente ........... ✅
     - Campos conhecidos: 22
     - Arquivos processados: 0
   • SeletorArquivo .................. ✅
     - Arquivo de dados: Cópia de xSAPtemp4687_JAN_25.xls
     - Tipo: .XLS

💡 Próximo passo:
   BLOCO 4 vai detectar o cabeçalho e carregar os dados

Digite 'BLOCO 3 OK' para prosseguir ao BLOCO 4


In [45]:
# ═══════════════════════════════════════════════════════════════════
# CLASSE BASE PARA GUIs COM TIMER (como no prompt UX/UI)
# ═══════════════════════════════════════════════════════════════════

class GUIComTimer:
    """
    Implementa timer de 10s com countdown visual
    Baseado no prompt interno UX/UI
    """

    @staticmethod
    def criar_janela_com_timer(titulo, largura, altura, tem_timer=True):
        """Cria janela base com timer"""
        root = tk.Tk()
        root.title(titulo)
        root.geometry(f"{largura}x{altura}")
        root.resizable(False, False)

        # Centralizar
        x = (root.winfo_screenwidth() // 2) - (largura // 2)
        y = (root.winfo_screenheight() // 2) - (altura // 2)
        root.geometry(f"+{x}+{y}")
        root.attributes('-topmost', True)
        root.after(100, lambda: root.attributes('-topmost', False))

        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        # Variáveis de controle
        resultado = {'valor': None, 'cancelado': False, 'timeout': False}
        contador = [10] if tem_timer else [0]

        return root, frame, resultado, contador

    @staticmethod
    def adicionar_timer(frame, root, resultado, contador):
        """Adiciona timer visual"""
        label_timer = tk.Label(
            frame,
            text=f"⏱️  {contador[0]}s",
            font=('Arial', 16, 'bold'),
            fg='#FF4444',
            bg='white'
        )
        label_timer.pack(pady=(5, 15))

        def countdown():
            if contador[0] > 0 and not resultado['cancelado']:
                contador[0] -= 1
                label_timer.config(text=f"⏱️  {contador[0]}s")
                root.after(1000, countdown)
            elif contador[0] == 0 and not resultado['cancelado']:
                resultado['timeout'] = True
                root.quit()
                root.destroy()

        return countdown

    @staticmethod
    def criar_botoes(frame, cmd_principal, cmd_secundario=None,
                     label_principal="Confirmar", label_secundario="Usar Último"):
        """Cria botões padronizados"""
        tk.Frame(frame, height=2, bg='#CCCCCC').pack(fill=tk.X, pady=10)

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=10)

        tk.Button(
            frame_btns,
            text=label_principal,
            command=cmd_principal,
            width=18,
            height=2,
            bg='#4CAF50',
            fg='white',
            font=('Arial', 10, 'bold'),
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=5)

        if cmd_secundario:
            tk.Button(
                frame_btns,
                text=label_secundario,
                command=cmd_secundario,
                width=18,
                height=2,
                bg='#2196F3',
                fg='white',
                font=('Arial', 10),
                cursor='hand2'
            ).pack(side=tk.LEFT, padx=5)

print("✅ Sistema de GUI com Timer carregado")

✅ Sistema de GUI com Timer carregado


In [27]:
# ═══════════════════════════════════════════════════════════════════
# SELEÇÃO DE ARQUIVO - SUPORTE MULTI-FORMATO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📂 SELEÇÃO DE ARQUIVO")
print("="*70)

# Carregar última seleção
config_file = fm.pastas['logs'] / '.ultimo_arquivo.json'
ultimo_arquivo = None
sessao_atual = False

if config_file.exists():
    try:
        with open(config_file, 'r', encoding='utf-8') as f:
            config = json.load(f)

        # Verificar se é da sessão atual (última hora)
        try:
            ts_config = datetime.fromisoformat(config.get('timestamp', ''))
            ts_agora = datetime.now()
            diff_minutos = (ts_agora - ts_config).total_seconds() / 60

            if diff_minutos < 60:  # Última hora = mesma sessão
                caminho_salvo = config.get('caminho')
                if caminho_salvo and Path(caminho_salvo).exists():
                    ultimo_arquivo = Path(caminho_salvo)
                    sessao_atual = True
        except:
            pass
    except:
        pass

print(f"💡 Última seleção: {ultimo_arquivo.name if ultimo_arquivo else 'Nenhuma'}")
print(f"   Mesma sessão: {'Sim' if sessao_atual else 'Não'}")

# CASO 1: Tem arquivo da sessão atual → GUI com timer
if ultimo_arquivo and sessao_atual:
    def selecionar_arquivo_com_timer(ultimo_path):
        """GUI com timer para confirmar ou trocar arquivo"""
        root, frame, resultado, contador = GUIComTimer.criar_janela_com_timer(
            "DETECTOR - Seleção de Arquivo",
            650, 500,  # ✅ Aumentado altura para preview
            tem_timer=True
        )

        # Título
        tk.Label(
            frame,
            text="📂 Seleção de Arquivo",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        # Info do último arquivo
        tk.Label(
            frame,
            text="💡 Último arquivo selecionado nesta sessão:",
            font=('Arial', 10),
            bg='white'
        ).pack(pady=(0, 5))

        # ✅ DETECTAR TIPO DE ARQUIVO
        extensao = ultimo_path.suffix.lower()
        if extensao in ['.xlsx', '.xls', '.xlsm']:
            tipo_arquivo = "Excel"
            icone = "📊"
        elif extensao == '.csv':
            tipo_arquivo = "CSV"
            icone = "📄"
        elif extensao == '.txt':
            tipo_arquivo = "TXT"
            icone = "📝"
        else:
            tipo_arquivo = "Desconhecido"
            icone = "❓"

        # Box com info do arquivo
        frame_info = tk.Frame(frame, bg='#E3F2FD', relief=tk.SUNKEN, borderwidth=2)
        frame_info.pack(fill=tk.X, pady=(0, 10), padx=10)

        tk.Label(
            frame_info,
            text=f"{icone} {ultimo_path.name}",
            font=('Arial', 10, 'bold'),
            bg='#E3F2FD',
            fg='#1565C0',
            anchor='w'
        ).pack(fill=tk.X, padx=10, pady=(5, 2))

        tk.Label(
            frame_info,
            text=f"📦 Tipo: {tipo_arquivo} | 📏 Tamanho: {ultimo_path.stat().st_size / 1024:.1f} KB",
            font=('Arial', 9),
            bg='#E3F2FD',
            fg='#1565C0',
            anchor='w'
        ).pack(fill=tk.X, padx=10, pady=(0, 2))

        tk.Label(
            frame_info,
            text=f"📂 Local: {ultimo_path.parent}",
            font=('Arial', 9),
            bg='#E3F2FD',
            fg='#1565C0',
            anchor='w',
            wraplength=600
        ).pack(fill=tk.X, padx=10, pady=(0, 5))

        # Timer
        countdown = GUIComTimer.adicionar_timer(frame, root, resultado, contador)

        # Mensagem
        tk.Label(
            frame,
            text="Deseja usar este arquivo ou escolher outro?",
            font=('Arial', 10),
            bg='white'
        ).pack(pady=(10, 10))

        # ✅ PREVIEW INTELIGENTE POR TIPO
        try:
            frame_preview = tk.Frame(frame, bg='#F5F5F5', relief=tk.SUNKEN, borderwidth=1)
            frame_preview.pack(fill=tk.X, padx=10, pady=(0, 10))

            tk.Label(
                frame_preview,
                text="📊 Preview (3 primeiras linhas):",
                font=('Arial', 9, 'bold'),
                bg='#F5F5F5',
                anchor='w'
            ).pack(fill=tk.X, padx=5, pady=(3, 2))

            # Carregar preview conforme tipo
            if extensao in ['.xlsx', '.xls', '.xlsm']:
                df_quick = pd.read_excel(ultimo_path, nrows=3)
            elif extensao == '.csv':
                # Tentar diferentes encodings e separadores
                for encoding in ['utf-8', 'latin-1', 'cp1252']:
                    try:
                        df_quick = pd.read_csv(ultimo_path, nrows=3, encoding=encoding)
                        break
                    except:
                        continue
            elif extensao == '.txt':
                # Tentar como CSV com separadores comuns
                for sep in ['\t', ';', '|', ',']:
                    try:
                        df_quick = pd.read_csv(ultimo_path, nrows=3, sep=sep)
                        if len(df_quick.columns) > 1:  # Achou separador correto
                            break
                    except:
                        continue

            preview_text = f"{len(df_quick)} linhas × {len(df_quick.columns)} colunas"
            tk.Label(
                frame_preview,
                text=preview_text,
                font=('Arial', 8),
                bg='#F5F5F5',
                fg='#666666',
                anchor='w'
            ).pack(fill=tk.X, padx=5, pady=(0, 3))

        except Exception as e:
            # Preview falhou - não mostrar nada
            pass

        # Funções
        def usar_ultimo():
            resultado['cancelado'] = True
            resultado['valor'] = ultimo_path
            root.quit()
            root.destroy()

        def escolher_novo():
            resultado['cancelado'] = True
            root.withdraw()

            # ✅ FILETYPES EXPANDIDOS
            arquivo = filedialog.askopenfilename(
                title="Selecione o arquivo de dados",
                initialdir=ultimo_path.parent,
                filetypes=[
                    ("Todos os suportados", "*.xlsx *.xls *.xlsm *.csv *.txt"),
                    ("Excel", "*.xlsx *.xls *.xlsm"),
                    ("CSV", "*.csv"),
                    ("TXT", "*.txt"),
                    ("Todos", "*.*")
                ]
            )

            resultado['valor'] = Path(arquivo) if arquivo else ultimo_path
            root.quit()
            root.destroy()

        # Botões
        tk.Frame(frame, height=2, bg='#CCCCCC').pack(fill=tk.X, pady=10)

        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=10)

        tk.Button(
            frame_btns,
            text="Escolher Novo Arquivo",
            command=escolher_novo,
            width=22,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white',
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=5)

        tk.Button(
            frame_btns,
            text=f"Usar '{ultimo_path.name[:15]}...' (10s)",
            command=usar_ultimo,
            width=30,
            height=2,
            font=('Arial', 10),
            bg='#2196F3',
            fg='white',
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=5)

        # Iniciar timer
        root.after(1000, countdown)
        root.mainloop()

        # Processar resultado
        if resultado.get('timeout'):
            print(f"   ⏱️  Timeout (10s) - usando último arquivo")
            return ultimo_path

        return resultado['valor']

    # Executar GUI
    print("\nAbrindo janela...")
    arquivo_selecionado = selecionar_arquivo_com_timer(ultimo_arquivo)

# CASO 2: Não tem arquivo da sessão → GUI direta sem timer
else:
    def selecionar_arquivo_direto():
        """GUI direta para primeira seleção"""
        root = tk.Tk()
        root.withdraw()
        root.attributes('-topmost', True)

        # ✅ FILETYPES EXPANDIDOS
        arquivo = filedialog.askopenfilename(
            title="Selecione o arquivo de dados",
            initialdir=ultimo_arquivo.parent if ultimo_arquivo else fm.pastas['entrada'],
            filetypes=[
                ("Todos os suportados", "*.xlsx *.xls *.xlsm *.csv *.txt"),
                ("Excel", "*.xlsx *.xls *.xlsm"),
                ("CSV", "*.csv"),
                ("TXT (Tabelas)", "*.txt"),
                ("Todos", "*.*")
            ]
        )

        root.destroy()

        if not arquivo:
            raise ValueError("❌ Nenhum arquivo selecionado")

        return Path(arquivo)

    print("\nAbrindo janela de seleção...")
    print("(A janela pode estar atrás do navegador)")
    arquivo_selecionado = selecionar_arquivo_direto()

# ✅ DETECTAR TIPO DE ARQUIVO
extensao = arquivo_selecionado.suffix.lower()
if extensao in ['.xlsx', '.xls', '.xlsm']:
    tipo_arquivo = "Excel"
elif extensao == '.csv':
    tipo_arquivo = "CSV"
elif extensao == '.txt':
    tipo_arquivo = "TXT"
else:
    tipo_arquivo = "Desconhecido"

# Salvar seleção no log (COM TIPO)
with open(config_file, 'w', encoding='utf-8') as f:
    json.dump({
        'nome': arquivo_selecionado.name,
        'caminho': str(arquivo_selecionado),
        'tamanho_kb': arquivo_selecionado.stat().st_size / 1024,
        'tipo': tipo_arquivo,  # ✅ NOVO
        'extensao': extensao,  # ✅ NOVO
        'timestamp': datetime.now().isoformat()
    }, f, indent=2)

# Exibir confirmação
print(f"\n✅ Arquivo selecionado:")
print(f"   📄 Nome: {arquivo_selecionado.name}")
print(f"   📦 Tipo: {tipo_arquivo}")
print(f"   📏 Tamanho: {arquivo_selecionado.stat().st_size / 1024:.1f} KB")
print(f"   📂 Pasta: {arquivo_selecionado.parent.name}")


📂 SELEÇÃO DE ARQUIVO
💡 Última seleção: Nenhuma
   Mesma sessão: Não

Abrindo janela de seleção...
(A janela pode estar atrás do navegador)

✅ Arquivo selecionado:
   📄 Nome: Cópia de xSAPtemp4687_JAN_25.xls
   📦 Tipo: Excel
   📏 Tamanho: 16257.5 KB
   📂 Pasta: Dado BW


In [28]:
# ═══════════════════════════════════════════════════════════════════
# CARREGAMENTO INTELIGENTE - EXCEL E CSV
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📥 CARREGAMENTO DO ARQUIVO")
print("="*70)

# Detectar tipo de arquivo pela extensão
extensao = arquivo_selecionado.suffix.lower()
print(f"\n🔍 Extensão detectada: {extensao}")

# ═══════════════════════════════════════════════════════════════════
# CASO 1: ARQUIVOS EXCEL (.xls, .xlsx, .xlsm)
# ═══════════════════════════════════════════════════════════════════

if extensao in ['.xls', '.xlsx', '.xlsm']:
    print(f"📊 Tipo: EXCEL")

    try:
        # Tentar xlrd primeiro (para .xls antigos)
        workbook = xlrd.open_workbook(str(arquivo_selecionado))
        sheets = workbook.sheet_names()
        metodo_carga = 'xlrd'
        print(f"   ✅ Método: xlrd (XLS)")

    except:
        # Se falhar, usar pandas (para .xlsx/.xlsm)
        try:
            workbook = pd.ExcelFile(str(arquivo_selecionado))
            sheets = workbook.sheet_names
            metodo_carga = 'pandas'
            print(f"   ✅ Método: pandas (XLSX/XLSM)")
        except Exception as e:
            print(f"   ❌ ERRO ao abrir Excel: {e}")
            raise

    print(f"\n📋 Sheets encontradas: {len(sheets)}")
    for i, sheet in enumerate(sheets, 1):
        print(f"   {i}. {sheet}")

    tipo_arquivo = 'EXCEL'

# ═══════════════════════════════════════════════════════════════════
# CASO 2: ARQUIVOS CSV (.csv)
# ═══════════════════════════════════════════════════════════════════

elif extensao == '.csv':
    print(f"📄 Tipo: CSV")

    try:
        # Ler primeira linha para detectar separador
        with open(arquivo_selecionado, 'r', encoding='cp1252') as f:
            primeira_linha = f.readline().strip()

        print(f"\n🔍 Primeira linha: {primeira_linha[:100]}")

        # Detectar separador
        separador_detectado = None

        # Caso 1: Linha explícita com "sep="
        if primeira_linha.lower().startswith('sep='):
            separador_detectado = primeira_linha.split('=')[1]
            skiprows_csv = 1  # Pular primeira linha
            print(f"   ✅ Separador explícito detectado: '{separador_detectado}'")

        # Caso 2: Tentar detectar automaticamente
        else:
            # Testar separadores comuns
            for sep in ['^', ';', ',', '\t', '|']:
                df_test = pd.read_csv(
                    arquivo_selecionado,
                    nrows=2,
                    sep=sep,
                    encoding='cp1252',
                    on_bad_lines='skip'
                )
                if len(df_test.columns) > 1:  # Achou separador válido
                    separador_detectado = sep
                    skiprows_csv = 0
                    print(f"   ✅ Separador auto-detectado: '{separador_detectado}'")
                    break

        if not separador_detectado:
            raise ValueError("❌ Não foi possível detectar o separador do CSV")

        # Carregar preview completo
        df_preview = pd.read_csv(
            arquivo_selecionado,
            sep=separador_detectado,
            encoding='cp1252',
            skiprows=skiprows_csv,
            nrows=5
        )

        print(f"\n📊 Estrutura do CSV:")
        print(f"   Colunas: {len(df_preview.columns)}")
        print(f"   Encoding: cp1252")
        print(f"   Separador: '{separador_detectado}'")

        # Simular comportamento de sheets (CSV tem apenas 1 "sheet")
        sheets = ['Dados CSV']  # Nome virtual
        metodo_carga = 'csv'
        workbook = None  # Não há workbook em CSV
        tipo_arquivo = 'CSV'

        print(f"\n📋 Sheet virtual criada: 'Dados CSV'")

    except Exception as e:
        print(f"   ❌ ERRO ao processar CSV: {e}")
        raise

# ═══════════════════════════════════════════════════════════════════
# CASO 3: ARQUIVOS TXT (.txt)
# ═══════════════════════════════════════════════════════════════════

elif extensao == '.txt':
    print(f"📝 Tipo: TXT")
    print(f"   ⚠️  Processamento similar a CSV")

    # Processar como CSV
    # [Mesmo código do CSV acima]

else:
    raise ValueError(f"❌ Formato não suportado: {extensao}")

# ═══════════════════════════════════════════════════════════════════
# RESUMO DO CARREGAMENTO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("✅ CARREGAMENTO CONCLUÍDO")
print("─"*70)
print(f"   Tipo: {tipo_arquivo}")
print(f"   Método: {metodo_carga}")
print(f"   Sheets/Tabelas: {len(sheets)}")


📥 CARREGAMENTO DO ARQUIVO

🔍 Extensão detectada: .xls
📊 Tipo: EXCEL
   ✅ Método: xlrd (XLS)

📋 Sheets encontradas: 11
   1. SAPBEXqueriesDefunct
   2. SAPBEXfiltersDefunct
   3. Valor da Variação Total
   4. Valor da Variação Total Grupo
   5. Limite Técnico
   6. Justificar
   7. Limite Técnico Grupo
   8. BExRepositorySheet
   9. Justificar Grupo
   10. Custo do Produto
   11. Imposto

──────────────────────────────────────────────────────────────────────
✅ CARREGAMENTO CONCLUÍDO
──────────────────────────────────────────────────────────────────────
   Tipo: EXCEL
   Método: xlrd
   Sheets/Tabelas: 11


In [29]:
# ═══════════════════════════════════════════════════════════════════
# SELEÇÃO DE SHEET (COM SUPORTE CSV)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📋 SELEÇÃO DE SHEET/TABELA")
print("="*70)

# Carregar última seleção
config_file = fm.pastas['logs'] / '.ultima_sheet.json'
ultima_sheet = None
arquivo_mudou = True

if config_file.exists():
    try:
        with open(config_file, 'r', encoding='utf-8') as f:
            config = json.load(f)
            # Verificar se é o mesmo arquivo E timestamp recente (última hora)
            if config.get('arquivo') == arquivo_selecionado.name:
                ultima_sheet = config.get('sheet')
                # Verificar se timestamp é recente
                try:
                    ts_salvo = datetime.fromisoformat(config.get('timestamp', ''))
                    ts_agora = datetime.now()
                    diff_minutos = (ts_agora - ts_salvo).total_seconds() / 60

                    if diff_minutos < 60:  # Última hora
                        arquivo_mudou = False
                except:
                    pass
    except:
        pass

print(f"\n💡 Última sheet: {ultima_sheet if ultima_sheet else 'Nenhuma'}")
print(f"   Arquivo mudou: {'Sim' if arquivo_mudou else 'Não'}")

# ═══════════════════════════════════════════════════════════════════
# CASO 1: CSV - SELEÇÃO AUTOMÁTICA (apenas 1 sheet virtual)
# ═══════════════════════════════════════════════════════════════════

if tipo_arquivo == 'CSV':
    sheet_nome = sheets[0]  # 'Dados CSV'
    print(f"\n✅ Arquivo CSV - usando sheet virtual automática: '{sheet_nome}'")

# ═══════════════════════════════════════════════════════════════════
# CASO 2: EXCEL - Apenas 1 sheet E arquivo mudou → Usar diretamente
# ═══════════════════════════════════════════════════════════════════

elif len(sheets) == 1 and arquivo_mudou:
    sheet_nome = sheets[0]
    print(f"\n✅ Apenas 1 sheet - selecionando automaticamente: '{sheet_nome}'")

# ═══════════════════════════════════════════════════════════════════
# CASO 3: EXCEL - Mais de 1 sheet OU tem histórico → GUI COM TIMER
# ═══════════════════════════════════════════════════════════════════

else:
    def selecionar_sheet_com_timer(sheets, ultima=None, mostrar_timer=True):
        """GUI com timer para seleção de sheet"""
        root, frame, resultado, contador = GUIComTimer.criar_janela_com_timer(
            "DETECTOR - Seleção de Sheet",
            600, 450,
            tem_timer=(mostrar_timer and ultima is not None)
        )

        # Título
        tk.Label(
            frame,
            text="📋 Seleção de Sheet",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 10))

        tk.Label(
            frame,
            text="Selecione a Sheet para processar:",
            font=('Arial', 12, 'bold'),
            bg='white'
        ).pack(pady=(0, 10))

        # Timer (se tem última E timer ativo)
        if ultima and mostrar_timer:
            tk.Label(
                frame,
                text=f"💡 Última sheet usada: '{ultima}'",
                font=('Arial', 10),
                bg='#E3F2FD',
                fg='#1565C0',
                padx=10,
                pady=10
            ).pack(fill=tk.X, pady=(0, 5))

            countdown = GUIComTimer.adicionar_timer(frame, root, resultado, contador)

        # Listbox
        frame_list = tk.Frame(frame, bg='white')
        frame_list.pack(fill=tk.BOTH, expand=True, pady=(0, 10))

        scrollbar = tk.Scrollbar(frame_list)
        scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        listbox = tk.Listbox(
            frame_list,
            yscrollcommand=scrollbar.set,
            font=('Arial', 10),
            height=8
        )
        listbox.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)
        scrollbar.config(command=listbox.yview)

        for sheet in sheets:
            listbox.insert(tk.END, sheet)

        # Selecionar última ou primeira
        if ultima and ultima in sheets:
            idx = sheets.index(ultima)
            listbox.select_set(idx)
            listbox.see(idx)
        else:
            listbox.select_set(0)

        # Funções
        def nova_selecao():
            resultado['cancelado'] = True
            selecao = listbox.curselection()
            if selecao:
                resultado['valor'] = sheets[selecao[0]]
            root.quit()
            root.destroy()

        def usar_ultima():
            resultado['cancelado'] = True
            resultado['valor'] = ultima
            root.quit()
            root.destroy()

        def duplo_clique(event):
            nova_selecao()

        listbox.bind('<Double-Button-1>', duplo_clique)

        # Botões
        GUIComTimer.criar_botoes(
            frame,
            nova_selecao,
            usar_ultima if (ultima and mostrar_timer) else None,
            "Selecionar",
            f"Usar '{ultima}' (10s)" if ultima else None
        )

        # Iniciar timer
        if ultima and mostrar_timer:
            root.after(1000, countdown)

        root.mainloop()

        # Processar resultado
        if resultado.get('timeout') and ultima:
            print(f"   ⏱️  Timeout (10s) - usando última sheet")
            return ultima

        return resultado['valor']

    # Executar GUI
    print(f"\nAbrindo janela de seleção...")
    sheet_nome = selecionar_sheet_com_timer(
        sheets,
        ultima_sheet,
        mostrar_timer=(not arquivo_mudou)  # Timer apenas se mesmo arquivo
    )

# ═══════════════════════════════════════════════════════════════════
# SALVAR ESCOLHA
# ═══════════════════════════════════════════════════════════════════

with open(config_file, 'w', encoding='utf-8') as f:
    json.dump({
        'arquivo': arquivo_selecionado.name,
        'sheet': sheet_nome,
        'timestamp': datetime.now().isoformat()
    }, f, indent=2)

print(f"\n✅ Sheet selecionada: '{sheet_nome}'")

# ═══════════════════════════════════════════════════════════════════
# RESUMO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("✅ SELEÇÃO CONCLUÍDA")
print("─"*70)
print(f"   Arquivo: {arquivo_selecionado.name}")
print(f"   Sheet: {sheet_nome}")
print(f"   Tipo: {tipo_arquivo}")


📋 SELEÇÃO DE SHEET/TABELA

💡 Última sheet: Nenhuma
   Arquivo mudou: Sim

Abrindo janela de seleção...

✅ Sheet selecionada: 'Valor da Variação Total'

──────────────────────────────────────────────────────────────────────
✅ SELEÇÃO CONCLUÍDA
──────────────────────────────────────────────────────────────────────
   Arquivo: Cópia de xSAPtemp4687_JAN_25.xls
   Sheet: Valor da Variação Total
   Tipo: EXCEL


In [36]:
# ═══════════════════════════════════════════════════════════════════
# PREVIEW VISUAL (50 linhas × 15 colunas) - SUPORTE CSV
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("👀 PREVIEW DO ARQUIVO")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# CASO 1: EXCEL com xlrd (arquivos .xls antigos)
# ═══════════════════════════════════════════════════════════════════

if metodo_carga == 'xlrd':
    print("📊 Método: xlrd")

    sheet = workbook.sheet_by_name(sheet_nome)
    data_preview = []

    for row_idx in range(min(50, sheet.nrows)):
        data_preview.append(sheet.row_values(row_idx))

    df_preview = pd.DataFrame(data_preview)

# ═══════════════════════════════════════════════════════════════════
# CASO 2: EXCEL com pandas (arquivos .xlsx/.xlsm)
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'pandas':
    print("📊 Método: pandas Excel")

    df_preview = pd.read_excel(
        workbook,
        sheet_name=sheet_nome,
        nrows=50,
        header=None
    )

# ═══════════════════════════════════════════════════════════════════
# CASO 3: CSV 🆕
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'csv':
    print("📄 Método: CSV")

    df_preview = pd.read_csv(
        arquivo_selecionado,
        sep=separador_detectado,
        encoding='cp1252',
        skiprows=skiprows_csv,
        nrows=50,
        header=None  # Sem cabeçalho por enquanto
    )

else:
    raise ValueError(f"❌ Método de carga desconhecido: {metodo_carga}")

# ═══════════════════════════════════════════════════════════════════
# LIMITAR A 15 COLUNAS PARA VISUALIZAÇÃO
# ═══════════════════════════════════════════════════════════════════

df_preview_limitado = df_preview.iloc[:, :20].copy()

# ═══════════════════════════════════════════════════════════════════
# EXIBIR INFORMAÇÕES
# ═══════════════════════════════════════════════════════════════════

print(f"\n📊 Dimensões do preview:")
print(f"   Total: {df_preview.shape[0]} linhas × {df_preview.shape[1]} colunas")
print(f"   Exibindo: {df_preview_limitado.shape[0]} linhas × {df_preview_limitado.shape[1]} colunas")

print(f"\n👁️  Preview (primeiras 50 linhas, até 100 colunas):")
print("─" * 80)

# Usar display ou print dependendo do ambiente
try:
    display(df_preview_limitado)
except NameError:
    print(df_preview_limitado.to_string())

print("─" * 80)


👀 PREVIEW DO ARQUIVO
📊 Método: xlrd

📊 Dimensões do preview:
   Total: 50 linhas × 60 colunas
   Exibindo: 50 linhas × 20 colunas

👁️  Preview (primeiras 50 linhas, até 100 colunas):
────────────────────────────────────────────────────────────────────────────────


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,,,,,,,,,,,,,,,,VARIAÇÃO %,LIMITE TÉCNICO,,FALTA,SOBRA
1,,,,,,,,,,,,,,,,,,,,
2,,,,,,,,,,,,,,,,LIMITE DE COMPETÊNCIA,DIRETORIA,PRES,N2,N3
3,,,,,,,,,,,,,,,,R$ por produto / mês,">60.000.000,00",30000000.0,1000000.0,300000.0
4,,,,,,,,,,,,,,,,,,,,
5,,,,,,,,,,,,,,,,,,,,
6,,,,,,,,,,,,Variação de Estoque por Centro - GPA,,,,,,,,
7,,,,,,,,,,,,,,,,,,,,
8,,,,,,,,,,,,Centro de lucro,,,,,,,,
9,,,,,,,,,,,,Centro,,,,,,,,


────────────────────────────────────────────────────────────────────────────────


In [37]:
# ═══════════════════════════════════════════════════════════════════
# DETECÇÃO E SELEÇÃO AVANÇADA DE CABEÇALHO - COMPLETO
# Com: Dicionário Persistente + Análise de Repetição + Multi-linha
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🎯 DETECÇÃO E SELEÇÃO DE CABEÇALHO")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# CARREGAR DICIONÁRIO PERSISTENTE (se não estiver carregado)
# ═══════════════════════════════════════════════════════════════════

if 'DICIONARIO_PERSISTENTE' not in globals():
    print("\n📚 Carregando dicionário persistente...")

    # Locais possíveis
    locais_dicionario = [
        Path.cwd() / 'DICT_Dicionario_Persistente.json',
        fm.pastas['logs'] / 'DICT_Dicionario_Persistente.json',
        Path.cwd().parent / 'DICT_Dicionario_Persistente.json',
    ]

    DICIONARIO_PERSISTENTE = None

    for local in locais_dicionario:
        if local.exists():
            try:
                with open(local, 'r', encoding='utf-8') as f:
                    DICIONARIO_PERSISTENTE = json.load(f)
                print(f"   ✅ Carregado de: {local.name}")
                break
            except:
                continue

    if not DICIONARIO_PERSISTENTE:
        print(f"   ℹ️  Dicionário não encontrado - criando vazio")
        DICIONARIO_PERSISTENTE = {
            'arquivos': {},
            'ultima_atualizacao': None,
            'versao': '1.0'
        }
else:
    print("\n📚 Dicionário persistente já carregado")

# ═══════════════════════════════════════════════════════════════════
# FUNÇÃO AVANÇADA DE AVALIAÇÃO
# ═══════════════════════════════════════════════════════════════════

def avaliar_linha_cabecalho_avancada(linha, idx, total_linhas, df_preview, dicionario_persistente):
    """
    Avalia linha como candidata a cabeçalho com múltiplas heurísticas.
    """
    celulas = [str(c).strip() for c in linha if str(c).strip() and str(c).strip().lower() not in ['nan', 'none', '']]

    if not celulas:
        return {'score': 0.0, 'detalhes': 'Linha vazia', 'matches_dicionario': []}

    score = 0.0
    detalhes = []

    # CRITÉRIO 1: PROPORÇÃO DE CÉLULAS PREENCHIDAS (peso 2.0)
    prop_preenchidas = len(celulas) / len(linha)
    score_preenchidas = prop_preenchidas * 2.0
    score += score_preenchidas
    detalhes.append(f"Preench: {prop_preenchidas:.0%} (+{score_preenchidas:.1f})")

    # CRITÉRIO 2: PROPORÇÃO DE TEXTO (peso 2.5)
    tem_texto = sum(1 for c in celulas if re.search(r'[a-zA-Z]', c))
    prop_texto = tem_texto / len(celulas) if celulas else 0
    score_texto = prop_texto * 2.5
    score += score_texto
    detalhes.append(f"Texto: {prop_texto:.0%} (+{score_texto:.1f})")

    # CRITÉRIO 3: MATCH COM DICIONÁRIO PERSISTENTE (peso 4.0)
    bonus_dicionario = 0.0
    matches_dicionario = []

    if dicionario_persistente and 'arquivos' in dicionario_persistente:
        campos_conhecidos = set()
        for arquivo_info in dicionario_persistente.get('arquivos', {}).values():
            if 'campos_mapeados' in arquivo_info:
                for campo_info in arquivo_info['campos_mapeados'].values():
                    if 'nome_original' in campo_info:
                        campos_conhecidos.add(campo_info['nome_original'].lower())
                    if 'nome_padrao' in campo_info:
                        campos_conhecidos.add(campo_info['nome_padrao'].lower())

        for celula in celulas:
            celula_lower = celula.lower()
            if celula_lower in campos_conhecidos:
                bonus_dicionario += 0.8
                matches_dicionario.append(celula[:20])
            elif any(conhecido in celula_lower for conhecido in campos_conhecidos):
                bonus_dicionario += 0.4
                matches_dicionario.append(f"{celula[:15]}*")

    bonus_dicionario = min(bonus_dicionario, 4.0)
    score += bonus_dicionario

    if bonus_dicionario > 0:
        detalhes.append(f"Dict: {len(matches_dicionario)}m (+{bonus_dicionario:.1f})")

    # CRITÉRIO 4: ANÁLISE DE REPETIÇÃO (peso 3.0)
    try:
        linhas_futuras = min(20, total_linhas - idx - 1)

        if linhas_futuras >= 5:
            colunas_com_repeticao = 0
            total_colunas_analisadas = 0

            for col_idx, valor_atual in enumerate(linha):
                valor_atual_str = str(valor_atual).strip()

                if not valor_atual_str or valor_atual_str.lower() in ['nan', 'none', '']:
                    continue

                total_colunas_analisadas += 1

                repeticoes = 0
                for i in range(1, min(linhas_futuras + 1, 21)):
                    if idx + i < len(df_preview):
                        valor_futuro = str(df_preview.iloc[idx + i, col_idx]).strip()
                        if valor_futuro == valor_atual_str:
                            repeticoes += 1

                if repeticoes >= 2:
                    colunas_com_repeticao += 1

            if total_colunas_analisadas > 0:
                prop_repeticao = colunas_com_repeticao / total_colunas_analisadas
                score_repeticao = (1 - prop_repeticao) * 3.0
                score += score_repeticao
                detalhes.append(f"Unic: {(1-prop_repeticao):.0%} (+{score_repeticao:.1f})")
    except:
        pass

    # CRITÉRIO 5: TAMANHO MÉDIO DE STRINGS (peso 1.0)
    tamanho_medio = np.mean([len(c) for c in celulas]) if celulas else 0
    if 5 <= tamanho_medio <= 50:
        score += 1.0
        detalhes.append(f"Tam: {tamanho_medio:.0f} (+1.0)")

    # CRITÉRIO 6: UNICIDADE DENTRO DA LINHA (peso 1.5)
    if len(celulas) == len(set(celulas)):
        score += 1.5
        detalhes.append("Únicos (+1.5)")

    # CRITÉRIO 7: POSIÇÃO NO ARQUIVO (peso 0.5)
    if idx < 50:
        bonus_posicao = (50 - idx) / 100
        score += bonus_posicao
        detalhes.append(f"Pos: {idx+1} (+{bonus_posicao:.2f})")

    return {
        'score': score,
        'detalhes': ' | '.join(detalhes),
        'matches_dicionario': matches_dicionario
    }

# ═══════════════════════════════════════════════════════════════════
# AVALIAR TODAS AS LINHAS
# ═══════════════════════════════════════════════════════════════════

print("\n📊 Analisando linhas para detectar cabeçalho...")

# Preparar dados para análise
if metodo_carga == 'csv':
    data_para_analise = df_preview.values.tolist()
elif metodo_carga == 'xlrd':
    data_para_analise = []
    sheet = workbook.sheet_by_name(sheet_nome)
    for row_idx in range(min(50, sheet.nrows)):
        data_para_analise.append(sheet.row_values(row_idx))
else:
    data_para_analise = df_preview.values.tolist()

scores = []

for idx, linha in enumerate(data_para_analise):
    resultado = avaliar_linha_cabecalho_avancada(
        linha,
        idx,
        len(data_para_analise),
        df_preview,
        DICIONARIO_PERSISTENTE
    )

    scores.append({
        'linha_excel': idx + 1,
        'indice': idx,
        'score': resultado['score'],
        'detalhes': resultado['detalhes'],
        'matches': resultado['matches_dicionario']
    })

scores = sorted(scores, key=lambda x: x['score'], reverse=True)

# ═══════════════════════════════════════════════════════════════════
# EXIBIR TOP 5 CANDIDATOS
# ═══════════════════════════════════════════════════════════════════

print("\n🏆 Top 5 candidatos a cabeçalho:")
for i, item in enumerate(scores[:5], 1):
    print(f"\n   {i}. Linha {item['linha_excel']} (Excel) = Índice {item['indice']} (Python)")
    print(f"      Score: {item['score']:.2f}/15")
    print(f"      {item['detalhes']}")
    if item['matches']:
        print(f"      Matches: {', '.join(item['matches'][:5])}")

melhor = scores[0]
print(f"\n🎯 Sugestão automática: Linha {melhor['linha_excel']} (confiança {melhor['score']:.2f}/15)")

# ═══════════════════════════════════════════════════════════════════
# DETECÇÃO DE CABEÇALHO MULTI-LINHA
# ═══════════════════════════════════════════════════════════════════

print("\n🔍 Verificando cabeçalho multi-linha...")

if len(scores) > 1:
    segundo = scores[1]

    if segundo['indice'] == melhor['indice'] + 1 and segundo['score'] > (melhor['score'] * 0.5):
        print(f"   ⚠️  POSSÍVEL CABEÇALHO MULTI-LINHA detectado!")
        print(f"   Linha {melhor['linha_excel']}: Score {melhor['score']:.2f}")
        print(f"   Linha {segundo['linha_excel']}: Score {segundo['score']:.2f}")
        print(f"\n   💡 RECOMENDAÇÃO:")
        print(f"      1. CONCATENAR: Linha1 + ' - ' + Linha2")
        print(f"      2. USAR PRIMEIRA: Linha {melhor['linha_excel']}")
        print(f"      3. PERSONALIZAR via GUI")

        multi_linha_detectado = True
        linha_fim_sugerida = segundo['linha_excel']
    else:
        multi_linha_detectado = False
        linha_fim_sugerida = melhor['linha_excel']
else:
    multi_linha_detectado = False
    linha_fim_sugerida = melhor['linha_excel']

# ═══════════════════════════════════════════════════════════════════
# GUI AVANÇADA COM TIMER
# ═══════════════════════════════════════════════════════════════════

def selecionar_range_cabecalho_com_timer(sugerido_linha, sugerido_linha_fim, total_linhas, total_colunas, multi_linha=False):
    """GUI avançada para seleção de range de cabeçalho."""

    config_file = fm.pastas['logs'] / '.ultimo_range_cabecalho.json'
    ultimo_config = None

    if config_file.exists():
        try:
            with open(config_file, 'r', encoding='utf-8') as f:
                cfg = json.load(f)
                if cfg.get('arquivo') == arquivo_selecionado.name:
                    ultimo_config = cfg
        except:
            pass

    root, frame, resultado, contador = GUIComTimer.criar_janela_com_timer(
        "DETECTOR - Seleção Avançada de Cabeçalho",
        650, 650,
        tem_timer=bool(ultimo_config)
    )

    tk.Label(
        frame,
        text="📋 Configuração de Cabeçalho e Dados",
        font=('Arial', 14, 'bold'),
        bg='white'
    ).pack(pady=(0, 15))

    texto_sugestao = f"🤖 Sugestão: Linha {sugerido_linha}"
    if multi_linha and sugerido_linha_fim != sugerido_linha:
        texto_sugestao += f" a {sugerido_linha_fim} (MULTI-LINHA)"

    tk.Label(
        frame,
        text=texto_sugestao,
        font=('Arial', 10),
        bg='#E8F5E9' if not multi_linha else '#FFF9C4',
        fg='#2E7D32' if not multi_linha else '#F57F17',
        padx=10,
        pady=8
    ).pack(fill=tk.X, pady=(0, 5))

    if ultimo_config:
        texto_ultimo = f"💡 Última config: Cabeçalho L{ultimo_config['linha_inicio']}"
        if ultimo_config.get('linha_fim', ultimo_config['linha_inicio']) != ultimo_config['linha_inicio']:
            texto_ultimo += f"-L{ultimo_config['linha_fim']}"
        texto_ultimo += f" | Col {ultimo_config['col_inicio']}-{ultimo_config['col_fim']}"

        tk.Label(
            frame,
            text=texto_ultimo,
            font=('Arial', 10),
            bg='#E3F2FD',
            fg='#1565C0',
            padx=10,
            pady=8
        ).pack(fill=tk.X, pady=(0, 10))

        countdown = GUIComTimer.adicionar_timer(frame, root, resultado, contador)

    frame_inputs = tk.Frame(frame, bg='white')
    frame_inputs.pack(fill=tk.X, pady=(10, 0))

    tk.Label(frame_inputs, text="Linha Cabeçalho INÍCIO (Excel):", bg='white', font=('Arial', 9)).grid(row=0, column=0, sticky='w', padx=5, pady=5)
    entry_cab_ini = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_cab_ini.insert(0, str(ultimo_config['linha_inicio'] if ultimo_config else sugerido_linha))
    entry_cab_ini.grid(row=0, column=1, padx=5, pady=5)

    tk.Label(frame_inputs, text="Linha Cabeçalho FIM (Excel):", bg='white', font=('Arial', 9)).grid(row=1, column=0, sticky='w', padx=5, pady=5)
    entry_cab_fim = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_cab_fim.insert(0, str(ultimo_config.get('linha_fim', sugerido_linha_fim) if ultimo_config else sugerido_linha_fim))
    entry_cab_fim.grid(row=1, column=1, padx=5, pady=5)

    tk.Label(frame_inputs, text="Coluna INÍCIO:", bg='white', font=('Arial', 9)).grid(row=2, column=0, sticky='w', padx=5, pady=5)
    entry_col_ini = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_col_ini.insert(0, str(ultimo_config['col_inicio'] if ultimo_config else 1))
    entry_col_ini.grid(row=2, column=1, padx=5, pady=5)

    tk.Label(frame_inputs, text="Coluna FIM:", bg='white', font=('Arial', 9)).grid(row=3, column=0, sticky='w', padx=5, pady=5)
    entry_col_fim = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    entry_col_fim.insert(0, str(ultimo_config['col_fim'] if ultimo_config else total_colunas))
    entry_col_fim.grid(row=3, column=1, padx=5, pady=5)

    tk.Label(frame_inputs, text="Linha DADOS início (Excel):", bg='white', font=('Arial', 9)).grid(row=4, column=0, sticky='w', padx=5, pady=5)
    entry_dados_ini = tk.Entry(frame_inputs, width=10, font=('Arial', 10))
    dados_inicio_default = (ultimo_config.get('linha_fim', sugerido_linha_fim) if ultimo_config else sugerido_linha_fim) + 1
    entry_dados_ini.insert(0, str(ultimo_config.get('linha_dados_inicio', dados_inicio_default) if ultimo_config else dados_inicio_default))
    entry_dados_ini.grid(row=4, column=1, padx=5, pady=5)

    tk.Label(
        frame,
        text="💡 Para cabeçalho 1 linha: Início = Fim\n💡 Para multi-linha: Início < Fim (ex: 3 a 4)",
        font=('Arial', 8, 'italic'),
        bg='#FFFDE7',
        fg='#F57F17',
        padx=10,
        pady=8,
        justify=tk.LEFT
    ).pack(fill=tk.X, pady=(10, 0))

    def confirmar():
        resultado['cancelado'] = True
        try:
            resultado['valor'] = {
                'linha_inicio': int(entry_cab_ini.get()),
                'linha_fim': int(entry_cab_fim.get()),
                'col_inicio': int(entry_col_ini.get()),
                'col_fim': int(entry_col_fim.get()),
                'linha_dados_inicio': int(entry_dados_ini.get())
            }
        except ValueError:
            resultado['valor'] = None
        root.quit()
        root.destroy()

    def usar_ultima():
        resultado['cancelado'] = True
        resultado['valor'] = {
            'linha_inicio': ultimo_config['linha_inicio'],
            'linha_fim': ultimo_config.get('linha_fim', ultimo_config['linha_inicio']),
            'col_inicio': ultimo_config['col_inicio'],
            'col_fim': ultimo_config['col_fim'],
            'linha_dados_inicio': ultimo_config['linha_dados_inicio']
        }
        root.quit()
        root.destroy()

    GUIComTimer.criar_botoes(
        frame,
        confirmar,
        usar_ultima if ultimo_config else None,
        "Confirmar",
        "Usar Última (10s)"
    )

    if ultimo_config:
        root.after(1000, countdown)

    root.mainloop()

    if resultado.get('timeout') and ultimo_config:
        print(f"   ⏱️  Timeout - usando última configuração")
        return {
            'linha_inicio': ultimo_config['linha_inicio'],
            'linha_fim': ultimo_config.get('linha_fim', ultimo_config['linha_inicio']),
            'col_inicio': ultimo_config['col_inicio'],
            'col_fim': ultimo_config['col_fim'],
            'linha_dados_inicio': ultimo_config['linha_dados_inicio']
        }

    return resultado['valor']

# ═══════════════════════════════════════════════════════════════════
# EXECUTAR GUI
# ═══════════════════════════════════════════════════════════════════

print("\nAbrindo janela de configuração...")
config = selecionar_range_cabecalho_com_timer(
    melhor['linha_excel'],
    linha_fim_sugerida,
    len(data_para_analise),
    len(data_para_analise[0]) if data_para_analise else 1,
    multi_linha=multi_linha_detectado
)

if not config:
    raise ValueError("❌ Configuração inválida ou cancelada")

# Salvar configuração
with open(fm.pastas['logs'] / '.ultimo_range_cabecalho.json', 'w', encoding='utf-8') as f:
    config_salvar = config.copy()
    config_salvar['arquivo'] = arquivo_selecionado.name
    config_salvar['sheet'] = sheet_nome
    config_salvar['timestamp'] = datetime.now().isoformat()
    json.dump(config_salvar, f, indent=2)

# Extrair informações
linha_cabecalho_inicio = config['linha_inicio']
linha_cabecalho_fim = config['linha_fim']
col_inicio = config['col_inicio']
col_fim = config['col_fim']
linha_dados_inicio = config['linha_dados_inicio']

print(f"\n✅ CONFIGURAÇÃO CONFIRMADA:")
print(f"   📋 Cabeçalho: Linhas {linha_cabecalho_inicio} a {linha_cabecalho_fim} (Excel)")
print(f"   📊 Colunas: {col_inicio} a {col_fim}")
print(f"   📈 Dados: A partir da linha {linha_dados_inicio} (Excel)")

# Converter para índices Python
idx_cab_inicio = linha_cabecalho_inicio - 1
idx_cab_fim = linha_cabecalho_fim - 1
idx_col_inicio = col_inicio - 1
idx_col_fim = col_fim
idx_dados_inicio = linha_dados_inicio - 1

print(f"\n🐍 Índices Python:")
print(f"   Cabeçalho: {idx_cab_inicio} a {idx_cab_fim}")
print(f"   Colunas: {idx_col_inicio} a {idx_col_fim}")
print(f"   Dados: a partir de {idx_dados_inicio}")

print("\n" + "="*70)
print("✅ DETECÇÃO DE CABEÇALHO CONCLUÍDA")
print("="*70)


🎯 DETECÇÃO E SELEÇÃO DE CABEÇALHO

📚 Carregando dicionário persistente...
   ℹ️  Dicionário não encontrado - criando vazio

📊 Analisando linhas para detectar cabeçalho...

🏆 Top 5 candidatos a cabeçalho:

   1. Linha 3 (Excel) = Índice 2 (Python)
      Score: 8.71/15
      Preench: 12% (+0.2) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 9 (+1.0) | Únicos (+1.5) | Pos: 3 (+0.48)

   2. Linha 1 (Excel) = Índice 0 (Python)
      Score: 8.63/15
      Preench: 7% (+0.1) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 8 (+1.0) | Únicos (+1.5) | Pos: 1 (+0.50)

   3. Linha 7 (Excel) = Índice 6 (Python)
      Score: 8.47/15
      Preench: 2% (+0.0) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 36 (+1.0) | Únicos (+1.5) | Pos: 7 (+0.44)

   4. Linha 9 (Excel) = Índice 8 (Python)
      Score: 8.45/15
      Preench: 2% (+0.0) | Texto: 100% (+2.5) | Unic: 100% (+3.0) | Tam: 15 (+1.0) | Únicos (+1.5) | Pos: 9 (+0.42)

   5. Linha 10 (Excel) = Índice 9 (Python)
      Score: 8.44/15
      Preen

In [38]:
# ═══════════════════════════════════════════════════════════════════
# EXTRAÇÃO DE DADOS COM RANGE CONFIGURADO (EXCEL E CSV)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📥 EXTRAÇÃO DE DADOS")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# CONVERSÃO DE ÍNDICES PARA USUÁRIO (EXCEL)
# ═══════════════════════════════════════════════════════════════════

print("\n📋 Configuração (notação Excel para usuário):")
print(f"   Cabeçalho: Linha(s) {linha_cabecalho_inicio} a {linha_cabecalho_fim}")
print(f"   Colunas: {col_inicio} a {col_fim} (Excel: A={chr(64+col_inicio)} a {chr(64+col_fim) if col_fim <= 26 else 'A'+chr(38+col_fim)})")
print(f"   Dados: A partir da linha {linha_dados_inicio}")

print(f"\n🐍 Índices Python (interno):")
print(f"   Cabeçalho: {idx_cab_inicio} a {idx_cab_fim}")
print(f"   Colunas: {idx_col_inicio} a {idx_col_fim}")
print(f"   Dados: a partir de {idx_dados_inicio}")

# ═══════════════════════════════════════════════════════════════════
# CASO 1: ARQUIVO CSV 🆕
# ═══════════════════════════════════════════════════════════════════

if metodo_carga == 'csv':
    print(f"\n📄 Método: CSV")

    try:
        # CASO 1A: Cabeçalho em 1 linha
        if linha_cabecalho_inicio == linha_cabecalho_fim:
            print(f"   📋 Cabeçalho: Linha única ({linha_cabecalho_inicio})")

            # Carregar com cabeçalho
            df = pd.read_csv(
                arquivo_selecionado,
                sep=separador_detectado,
                encoding='cp1252',
                skiprows=skiprows_csv,  # Pula "sep=^" se necessário
                header=idx_cab_inicio,  # Linha do cabeçalho
                usecols=range(idx_col_inicio, idx_col_fim)  # Colunas selecionadas
            )

            # Se há linhas entre cabeçalho e dados, pular
            linhas_pular = idx_dados_inicio - idx_cab_inicio - 1
            if linhas_pular > 0:
                print(f"   ⏭️  Pulando {linhas_pular} linha(s) após cabeçalho")
                df = df.iloc[linhas_pular:].copy()

        # CASO 1B: Cabeçalho multi-linha
        else:
            print(f"   📋 Cabeçalho: Multi-linha ({linha_cabecalho_inicio} a {linha_cabecalho_fim})")

            # Carregar sem cabeçalho
            df_temp = pd.read_csv(
                arquivo_selecionado,
                sep=separador_detectado,
                encoding='cp1252',
                skiprows=skiprows_csv,
                header=None,
                usecols=range(idx_col_inicio, idx_col_fim)
            )

            # Extrair linhas do cabeçalho
            cabecalho_linhas = df_temp.iloc[idx_cab_inicio:idx_cab_fim+1].values

            # Combinar linhas do cabeçalho
            cab_final = []
            for col_idx in range(cabecalho_linhas.shape[1]):
                partes = [
                    str(linha[col_idx]).strip()
                    for linha in cabecalho_linhas
                    if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
                ]
                cab_final.append(' - '.join(partes) if partes else f'Col_{col_idx}')

            # Extrair dados
            df = df_temp.iloc[idx_dados_inicio:].copy()
            df.columns = cab_final

        print(f"   ✅ Carregado: {len(df):,} registros × {len(df.columns)} colunas")

    except Exception as e:
        print(f"   ❌ ERRO ao extrair CSV: {str(e)}")
        print(f"\n   🔍 DEBUG:")
        print(f"      Arquivo: {arquivo_selecionado}")
        print(f"      Separador: '{separador_detectado}'")
        print(f"      Encoding: cp1252")
        print(f"      Skiprows: {skiprows_csv}")
        print(f"      Header: {idx_cab_inicio}")
        print(f"      Usecols: {idx_col_inicio} a {idx_col_fim}")
        raise

# ═══════════════════════════════════════════════════════════════════
# CASO 2: ARQUIVO EXCEL (PANDAS) - .xlsx/.xlsm
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'pandas':
    print(f"📋 Método: pandas (XLSX/XLSM)")

    try:
        # CASO 2A: Cabeçalho em 1 linha
        if linha_cabecalho_inicio == linha_cabecalho_fim:
            print(f"   📋 Cabeçalho: Linha única ({linha_cabecalho_inicio})")

            df = pd.read_excel(
                arquivo_selecionado,  # ⚠️ CORRIGIDO: usar arquivo_selecionado, não workbook
                sheet_name=sheet_nome,
                header=idx_cab_inicio,
                usecols=range(idx_col_inicio, idx_col_fim)
            )

            # Remover linhas antes dos dados (se header < linha_dados)
            linhas_pular = idx_dados_inicio - idx_cab_inicio - 1
            if linhas_pular > 0:
                print(f"   ⏭️  Pulando {linhas_pular} linha(s) após cabeçalho")
                df = df.iloc[linhas_pular:].copy()

        # CASO 2B: Cabeçalho multi-linha
        else:
            print(f"   📋 Cabeçalho: Multi-linha ({linha_cabecalho_inicio} a {linha_cabecalho_fim})")

            df_temp = pd.read_excel(
                arquivo_selecionado,
                sheet_name=sheet_nome,
                header=None,
                usecols=range(idx_col_inicio, idx_col_fim)
            )

            # Combinar linhas do cabeçalho
            cabecalho_linhas = df_temp.iloc[idx_cab_inicio:idx_cab_fim+1].values
            cab_final = []

            for col_idx in range(cabecalho_linhas.shape[1]):
                partes = [
                    str(linha[col_idx]).strip()
                    for linha in cabecalho_linhas
                    if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
                ]
                cab_final.append(' - '.join(partes) if partes else f'Col_{col_idx}')

            # Extrair dados
            df = df_temp.iloc[idx_dados_inicio:].copy()
            df.columns = cab_final

        print(f"   ✅ Carregado: {len(df):,} registros × {len(df.columns)} colunas")

    except Exception as e:
        print(f"   ❌ ERRO ao extrair com pandas: {str(e)}")
        print(f"\n   🔍 DEBUG:")
        print(f"      Arquivo: {arquivo_selecionado}")
        print(f"      Sheet: {sheet_nome}")
        print(f"      Cabeçalho: linhas {idx_cab_inicio} a {idx_cab_fim}")
        print(f"      Colunas: {idx_col_inicio} a {idx_col_fim}")
        print(f"      Dados: a partir de linha {idx_dados_inicio}")
        raise

# ═══════════════════════════════════════════════════════════════════
# CASO 3: ARQUIVO EXCEL (XLRD) - .xls antigos
# ═══════════════════════════════════════════════════════════════════

elif metodo_carga == 'xlrd':
    print(f"📋 Método: xlrd (XLS)")

    try:
        sheet = workbook.sheet_by_name(sheet_nome)

        # CASO 3A: Cabeçalho em 1 linha
        if linha_cabecalho_inicio == linha_cabecalho_fim:
            print(f"   📋 Cabeçalho: Linha única ({linha_cabecalho_inicio})")

            # Extrair cabeçalho
            cabecalho = sheet.row_values(idx_cab_inicio)[idx_col_inicio:idx_col_fim]

            # Extrair dados
            data = []
            for row_idx in range(idx_dados_inicio, sheet.nrows):
                data.append(sheet.row_values(row_idx)[idx_col_inicio:idx_col_fim])

            df = pd.DataFrame(data, columns=cabecalho)

        # CASO 3B: Cabeçalho multi-linha
        else:
            print(f"   📋 Cabeçalho: Multi-linha ({linha_cabecalho_inicio} a {linha_cabecalho_fim})")

            # Extrair linhas do cabeçalho
            cabecalho_linhas = []
            for linha_idx in range(idx_cab_inicio, idx_cab_fim + 1):
                cabecalho_linhas.append(sheet.row_values(linha_idx)[idx_col_inicio:idx_col_fim])

            # Combinar linhas do cabeçalho
            cab_final = []
            for col_idx in range(len(cabecalho_linhas[0])):
                partes = [
                    str(linha[col_idx]).strip()
                    for linha in cabecalho_linhas
                    if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
                ]
                cab_final.append(' - '.join(partes) if partes else f'Col_{col_idx}')

            # Extrair dados
            data = []
            for row_idx in range(idx_dados_inicio, sheet.nrows):
                data.append(sheet.row_values(row_idx)[idx_col_inicio:idx_col_fim])

            df = pd.DataFrame(data, columns=cab_final)

        print(f"   ✅ Carregado: {len(df):,} registros × {len(df.columns)} colunas")

    except Exception as e:
        print(f"   ❌ ERRO ao extrair com xlrd: {str(e)}")
        print(f"\n   🔍 DEBUG:")
        print(f"      Sheet: {sheet_nome}")
        print(f"      Cabeçalho: linhas {idx_cab_inicio} a {idx_cab_fim}")
        print(f"      Colunas: {idx_col_inicio} a {idx_col_fim}")
        print(f"      Dados: a partir de linha {idx_dados_inicio}")
        raise

else:
    raise ValueError(f"❌ Método de carga desconhecido: {metodo_carga}")

# ═══════════════════════════════════════════════════════════════════
# VALIDAÇÕES PÓS-EXTRAÇÃO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("✅ VALIDAÇÕES")
print("─"*70)

# Reset index
df = df.reset_index(drop=True)

# Validar shape
print(f"\n📊 Shape final:")
print(f"   Registros: {len(df):,}")
print(f"   Colunas: {len(df.columns)}")

# Validar colunas
print(f"\n📋 Primeiras 10 colunas:")
for i, col in enumerate(df.columns[:10], 1):
    print(f"   {i:2d}. {col}")

if len(df.columns) > 10:
    print(f"   ... e mais {len(df.columns) - 10} colunas")

# Validar registros
print(f"\n📈 Primeiras 5 linhas (amostra):")
print(df.head().to_string())

# Validar tipos
print(f"\n🔢 Tipos detectados:")
tipos_count = df.dtypes.value_counts()
for tipo, count in tipos_count.items():
    print(f"   {str(tipo).ljust(15)}: {count} coluna(s)")

# Validar nulos
print(f"\n⚠️  Valores nulos:")
nulos_total = df.isnull().sum().sum()
if nulos_total > 0:
    print(f"   Total: {nulos_total:,} células vazias")
    colunas_com_nulos = df.isnull().sum()
    colunas_com_nulos = colunas_com_nulos[colunas_com_nulos > 0].sort_values(ascending=False)
    print(f"\n   Top 5 colunas com nulos:")
    for col, count in colunas_com_nulos.head(5).items():
        pct = (count / len(df)) * 100
        print(f"      {col[:40].ljust(40)}: {count:>6,} ({pct:>5.1f}%)")
else:
    print(f"   ✅ Nenhum valor nulo!")

# Memória
memoria_mb = df.memory_usage(deep=True).sum() / 1024**2
print(f"\n💾 Memória utilizada: {memoria_mb:.2f} MB")

print("\n" + "="*70)
print("✅ EXTRAÇÃO CONCLUÍDA COM SUCESSO")
print("="*70)


📥 EXTRAÇÃO DE DADOS

📋 Configuração (notação Excel para usuário):
   Cabeçalho: Linha(s) 33 a 33
   Colunas: 12 a 60 (Excel: A=L a Ab)
   Dados: A partir da linha 34

🐍 Índices Python (interno):
   Cabeçalho: 32 a 32
   Colunas: 11 a 60
   Dados: a partir de 33
📋 Método: xlrd (XLS)
   📋 Cabeçalho: Linha única (33)
   ✅ Carregado: 968 registros × 49 colunas

──────────────────────────────────────────────────────────────────────
✅ VALIDAÇÕES
──────────────────────────────────────────────────────────────────────

📊 Shape final:
   Registros: 968
   Colunas: 49

📋 Primeiras 10 colunas:
    1. 
    2. 
    3. 
    4. 
    5. 
    6. 
    7. 
    8. 
    9. 
   10. 
   ... e mais 39 colunas

📈 Primeiras 5 linhas (amostra):
                                                                                                                                                                                                                                                                                

In [39]:
# ═══════════════════════════════════════════════════════════════════
# LIMPEZA DE ESTRUTURA
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🧹 LIMPEZA DE ESTRUTURA")
print("="*70)

df = df_bruto.copy()
log_limpeza = []

# 1. Colunas completamente vazias
colunas_vazias = df.columns[df.isna().all()].tolist()
if colunas_vazias:
    print(f"\n🗑️  Removendo {len(colunas_vazias)} colunas vazias:")
    for col in colunas_vazias[:5]:  # Mostrar apenas 5
        print(f"   • {col}")
    if len(colunas_vazias) > 5:
        print(f"   ... e mais {len(colunas_vazias) - 5}")

    df = df.drop(columns=colunas_vazias)
    log_limpeza.append(f"Removidas {len(colunas_vazias)} colunas vazias")

# 2. Linhas completamente vazias
linhas_vazias_antes = len(df)
df = df.dropna(how='all')
linhas_vazias = linhas_vazias_antes - len(df)

if linhas_vazias > 0:
    print(f"\n🗑️  Removidas {linhas_vazias} linhas completamente vazias")
    log_limpeza.append(f"Removidas {linhas_vazias} linhas vazias")

# 3. Limpar nomes de colunas
print(f"\n🧹 Limpando nomes de colunas...")
colunas_antes = df.columns.tolist()
colunas_limpas = []

for col in df.columns:
    # Limpar
    col_limpo = str(col).strip()
    col_limpo = col_limpo.lstrip("'\"")
    col_limpo = col_limpo.replace('\n', ' ').replace('\r', '')
    col_limpo = ' '.join(col_limpo.split())

    # Remover espaços extras
    col_limpo = re.sub(r'\s+', ' ', col_limpo)

    colunas_limpas.append(col_limpo)

# Contar modificações
modificados = sum(1 for orig, limpo in zip(colunas_antes, colunas_limpas) if orig != limpo)
if modificados > 0:
    print(f"   ✅ {modificados} nomes limpos")
    log_limpeza.append(f"Limpeza de nomes: {modificados} colunas")

df.columns = colunas_limpas

# 4. Renomear colunas duplicadas
contagem = Counter(colunas_limpas)
duplicadas = {c: n for c, n in contagem.items() if n > 1}

if duplicadas:
    print(f"\n⚠️  Renomeando {len(duplicadas)} colunas duplicadas:")
    colunas_finais = []
    contador = {}

    for col in colunas_limpas:
        if col in duplicadas:
            if col not in contador:
                contador[col] = 0
                colunas_finais.append(col)
            else:
                contador[col] += 1
                novo_nome = f"{col}_dup{contador[col]}"
                colunas_finais.append(novo_nome)
                print(f"   '{col}' → '{novo_nome}'")
        else:
            colunas_finais.append(col)

    df.columns = colunas_finais
    log_limpeza.append(f"Renomeadas {sum(contador.values())} colunas duplicadas")

# 5. Remover linhas de totais/resultados
padroes_remover = [
    r'(?i)^total',
    r'(?i)^resultado',
    r'(?i)^soma',
    r'(?i)^subtotal',
    r'(?i)^grand total',
    r'(?i)^média',
    r'(?i)^average'
]

linhas_remover = []
for idx, row in df.iterrows():
    primeira_celula = str(row.iloc[0]).strip().lower()
    if any(re.search(p, primeira_celula) for p in padroes_remover):
        linhas_remover.append(idx)

if linhas_remover:
    print(f"\n🗑️  Removendo {len(linhas_remover)} linhas de totais/resultados")
    df = df.drop(index=linhas_remover)
    log_limpeza.append(f"Removidas {len(linhas_remover)} linhas de totais")

# 6. Reset index
df = df.reset_index(drop=True)

# Criar cópia limpa
df_limpo = df.copy()

# Resumo
print(f"\n" + "="*70)
print(f"✅ LIMPEZA CONCLUÍDA")
print(f"="*70)
print(f"\n📊 Antes → Depois:")
print(f"   Registros: {len(df_bruto):,} → {len(df_limpo):,}")
print(f"   Colunas: {len(df_bruto.columns)} → {len(df_limpo.columns)}")

if log_limpeza:
    print(f"\n📝 Operações realizadas:")
    for i, op in enumerate(log_limpeza, 1):
        print(f"   {i}. {op}")
else:
    print(f"\n✅ Nenhuma limpeza necessária - dados já estavam limpos!")

print(f"\n👀 Preview dos dados limpos:")
print("─" * 80)
display(df_limpo.head(3))
print("─" * 80)


🧹 LIMPEZA DE ESTRUTURA


NameError: name 'df_bruto' is not defined

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# TRATAMENTO DE FORMATO BW/BEx - FORWARD FILL
# Baseado em: ETL - ESTOQUE E MOVIMENTAÇÃO (SAP BEx).ipynb - PASSO 4
# Posição: APÓS Limpeza de Estrutura, ANTES de Detecção de Campos
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔄 TRATAMENTO DE FORMATO BW/BEx (FORWARD FILL)")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# ETAPA 1: DETECTAR SE É FORMATO BW
# ═══════════════════════════════════════════════════════════════════

print("""
ℹ️  CONTEXTO:
   Arquivos do SAP Business Warehouse (BW/BEx) usam formatação hierárquica:
   - Dimensões aparecem apenas na PRIMEIRA linha de cada agrupamento
   - Linhas seguintes ficam VAZIAS até mudar o agrupamento

   Este bloco detecta automaticamente e aplica forward fill se necessário.
""")

print("\n" + "─"*70)
print("🔍 ETAPA 1: DETECTANDO FORMATO BW")
print("─"*70)

# Analisar primeiras colunas (geralmente dimensões em BW)
colunas_iniciais = df.columns[:min(6, len(df.columns))]

print(f"\n📊 Analisando {len(colunas_iniciais)} primeiras colunas:")

analise_colunas = []

for col in colunas_iniciais:
    # Contar valores não-nulos
    valores_nao_nulos = df[col].notna()
    count_nao_nulos = valores_nao_nulos.sum()

    # Contar valores únicos (ignorando NaN)
    valores_unicos = df[col].dropna().nunique()

    # Percentual de NaN
    pct_nulos = (len(df) - count_nao_nulos) / len(df) * 100

    # Armazenar análise
    analise_colunas.append({
        'coluna': col,
        'count_nao_nulos': count_nao_nulos,
        'valores_unicos': valores_unicos,
        'pct_nulos': pct_nulos,
        'eh_dimensao_bw': pct_nulos > 30 and valores_unicos < 1000
    })

    emoji = "🔄" if pct_nulos > 30 and valores_unicos < 1000 else "  "
    print(f"   {emoji} {col[:35].ljust(35)} | Únicos: {valores_unicos:>5} | NaN: {pct_nulos:>5.1f}%")

# Contar quantas colunas parecem ser BW
colunas_bw = [a for a in analise_colunas if a['eh_dimensao_bw']]

print(f"\n📊 RESULTADO DA DETECÇÃO:")
print(f"   Colunas com padrão BW: {len(colunas_bw)}/{len(colunas_iniciais)}")

eh_formato_bw = len(colunas_bw) >= 2  # Pelo menos 2 colunas com padrão BW

if eh_formato_bw:
    print(f"   ✅ FORMATO BW DETECTADO - Forward fill será aplicado")
else:
    print(f"   ℹ️  Formato padrão - Forward fill NÃO necessário")

# ═══════════════════════════════════════════════════════════════════
# ETAPA 2: APLICAR FFILL (SE DETECTADO)
# ═══════════════════════════════════════════════════════════════════

if eh_formato_bw:
    print("\n" + "─"*70)
    print("🔄 ETAPA 2: APLICANDO FORWARD FILL")
    print("─"*70)

    # Listar colunas que receberão ffill
    colunas_para_ffill = [a['coluna'] for a in analise_colunas if a['eh_dimensao_bw']]

    print(f"\n📋 Colunas que receberão forward fill ({len(colunas_para_ffill)}):")
    for i, col in enumerate(colunas_para_ffill, 1):
        print(f"   {i}. {col}")

    # Contar NULLs ANTES do ffill
    print(f"\n📊 Contagem de NULLs ANTES do ffill:")
    nulls_antes = {}
    total_nulls_antes = 0

    for col in colunas_para_ffill:
        count = int(df[col].isnull().sum())
        nulls_antes[col] = count
        total_nulls_antes += count
        print(f"   {col[:40].ljust(40)}: {count:>8,} NULLs")

    print(f"   {'TOTAL'.ljust(40)}: {total_nulls_antes:>8,} NULLs")

    # APLICAR FFILL
    print(f"\n🔄 Aplicando forward fill...")
    df[colunas_para_ffill] = df[colunas_para_ffill].ffill()

    # Contar NULLs DEPOIS do ffill
    print(f"\n✅ Contagem de NULLs DEPOIS do ffill:")
    total_nulls_depois = 0

    for col in colunas_para_ffill:
        nulls_depois = int(df[col].isnull().sum())
        total_nulls_depois += nulls_depois

        # Calcular preenchidas
        preenchidas = nulls_antes[col] - nulls_depois

        print(f"   {col[:40].ljust(40)}: {nulls_antes[col]:>8,} → {nulls_depois:>8,} (Δ {preenchidas:>7,})")

    total_preenchidas = total_nulls_antes - total_nulls_depois
    print(f"   {'TOTAL'.ljust(40)}: {total_nulls_antes:>8,} → {total_nulls_depois:>8,} (Δ {total_preenchidas:>7,})")

    # ═══════════════════════════════════════════════════════════════
    # ETAPA 3: VALIDAR RESULTADO
    # ═══════════════════════════════════════════════════════════════

    print("\n" + "─"*70)
    print("✅ ETAPA 3: VALIDAÇÃO DO RESULTADO")
    print("─"*70)

    print(f"\n👁️  COMPARAÇÃO (primeiras 10 linhas):")

    # Criar DataFrame comparativo
    print(f"\n   ANTES do ffill:")
    # Recarregar apenas para mostrar (não aplicar)
    if metodo_carga == 'csv':
        df_antes = pd.read_csv(
            arquivo_selecionado,
            sep=separador_detectado,
            encoding='cp1252',
            skiprows=skiprows_csv if 'skiprows_csv' in locals() else None,
            header=idx_cab_inicio,
            usecols=range(idx_col_inicio, idx_col_fim)
        )
        if idx_dados_inicio > idx_cab_inicio + 1:
            df_antes = df_antes.iloc[idx_dados_inicio - idx_cab_inicio - 1:]
    else:
        df_antes = pd.read_excel(
            arquivo_selecionado,
            sheet_name=sheet_nome if 'sheet_nome' in locals() else 0,
            header=idx_cab_inicio,
            usecols=range(idx_col_inicio, idx_col_fim)
        )
        if idx_dados_inicio > idx_cab_inicio + 1:
            df_antes = df_antes.iloc[idx_dados_inicio - idx_cab_inicio - 1:]

    print(df_antes[colunas_para_ffill].head(10).to_string())

    print(f"\n   DEPOIS do ffill:")
    print(df[colunas_para_ffill].head(10).to_string())

    # Verificar se primeira linha tem valores
    primeira_linha_nulos = df.iloc[0][colunas_para_ffill].isnull().sum()

    if primeira_linha_nulos > 0:
        print(f"\n⚠️  ATENÇÃO: Primeira linha ainda tem {primeira_linha_nulos} NaN(s)!")
        print(f"   Isso pode indicar problema no cabeçalho ou dados")
        print(f"   Recomenda-se revisar a extração de dados")
    else:
        print(f"\n✅ Primeira linha OK - Todos os valores preenchidos")

    print("\n" + "="*70)
    print("✅ FORWARD FILL APLICADO COM SUCESSO")
    print("="*70)

    print(f"\n📊 RESUMO:")
    print(f"   Colunas processadas: {len(colunas_para_ffill)}")
    print(f"   Células preenchidas: {total_preenchidas:,}")
    print(f"   Redução de NaN: {(total_preenchidas / total_nulls_antes * 100):.1f}%")

else:
    print("\n" + "="*70)
    print("ℹ️  FORWARD FILL NÃO NECESSÁRIO")
    print("="*70)
    print(f"\n   Arquivo não possui formatação BW/BEx hierárquica")
    print(f"   Prosseguindo para próxima etapa...")

print("\n" + "="*70)
print("✅ TRATAMENTO BW CONCLUÍDO")
print("="*70)

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# REMOÇÃO DE LINHAS DE TOTALIZAÇÃO (ARQUIVOS BW/BEx)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🗑️  REMOÇÃO DE LINHAS DE TOTALIZAÇÃO")
print("="*70)

print("\nℹ️  CONTEXTO:")
print("   Arquivos BW/BEx frequentemente contêm linhas de:")
print("   - Totais gerais")
print("   - Subtotais por agrupamento")
print("   - Resultados parciais")
print("   - Médias agregadas")
print("   Estas linhas INFLAM valores e devem ser removidas.\n")

# ───────────────────────────────────────────────────────────────────
# ETAPA 1: DETECTAR LINHAS DE TOTALIZAÇÃO
# ───────────────────────────────────────────────────────────────────

print("─" * 80)
print("🔍 ETAPA 1: DETECTANDO LINHAS DE TOTALIZAÇÃO")
print("─" * 80)

# Padrões para identificar totalizações
padroes_totalizacao = [
    # Português
    r'(?i)^total\b',
    r'(?i)^resultado\b',
    r'(?i)^soma\b',
    r'(?i)^subtotal\b',
    r'(?i)^parcial\b',
    r'(?i)^grand total',
    r'(?i)^média\b',
    r'(?i)^media\b',
    r'(?i)^consolidado\b',
    r'(?i)^geral\b',
    r'(?i)total geral',
    r'(?i)resultado final',

    # Inglês (comum em exports SAP)
    r'(?i)^overall',
    r'(?i)^average',
    r'(?i)^sum\b',
    r'(?i)^total\s',
    r'(?i)^result\b',

    # Padrões numéricos (ex: "Total 5262", "Resultado 1234")
    r'(?i)^total\s+\d+',
    r'(?i)^resultado\s+\d+',
    r'(?i)^subtotal\s+\d+',
]

# Compilar regex
padroes_compilados = [re.compile(p) for p in padroes_totalizacao]

# Função para verificar se linha é totalização
def eh_linha_totalizacao(row):
    """Verifica se linha é de totalização."""
    # Verificar primeira coluna (mais comum)
    primeira_celula = str(row.iloc[0]).strip()

    if any(padrao.search(primeira_celula) for padrao in padroes_compilados):
        return True

    # Verificar segunda coluna (caso primeira esteja preenchida com código)
    if len(row) > 1:
        segunda_celula = str(row.iloc[1]).strip()
        if any(padrao.search(segunda_celula) for padrao in padroes_compilados):
            return True

    # Verificar se TODAS as colunas categóricas estão vazias
    # (indicativo de linha de totalização em hierarquia BW)
    colunas_categoricas = df.select_dtypes(include=['object']).columns
    if len(colunas_categoricas) > 0:
        valores_cat = row[colunas_categoricas].dropna()
        # Se menos de 30% preenchidos E primeira célula sugere total
        if len(valores_cat) < len(colunas_categoricas) * 0.3:
            primeira_palavra = primeira_celula.lower().split()[0] if primeira_celula else ''
            if primeira_palavra in ['total', 'resultado', 'soma', 'média', 'media', 'geral']:
                return True

    return False

# Identificar linhas a remover
print(f"\n📊 Analisando {len(df):,} linhas...")

linhas_totalizacao = []
for idx, row in df.iterrows():
    if eh_linha_totalizacao(row):
        linhas_totalizacao.append(idx)

print(f"\n🔍 Linhas de totalização detectadas: {len(linhas_totalizacao)}")

if len(linhas_totalizacao) > 0:
    # Mostrar exemplos
    print(f"\n📋 Exemplos de linhas detectadas (primeiras 5):")
    for i, idx in enumerate(linhas_totalizacao[:5], 1):
        primeira_col = df.iloc[idx, 0]
        segunda_col = df.iloc[idx, 1] if len(df.columns) > 1 else 'N/A'
        print(f"   {i}. Linha {idx}: '{primeira_col}' | '{segunda_col}'")

    if len(linhas_totalizacao) > 5:
        print(f"   ... e mais {len(linhas_totalizacao) - 5}")

# ───────────────────────────────────────────────────────────────────
# ETAPA 2: REMOVER LINHAS DETECTADAS
# ───────────────────────────────────────────────────────────────────

if len(linhas_totalizacao) > 0:
    print("\n" + "─" * 80)
    print("🗑️  ETAPA 2: REMOVENDO LINHAS DE TOTALIZAÇÃO")
    print("─" * 80)

    # Backup
    registros_antes = len(df)

    # Remover
    df_sem_totais = df.drop(index=linhas_totalizacao)

    # Reset index
    df_sem_totais = df_sem_totais.reset_index(drop=True)

    registros_depois = len(df_sem_totais)
    removidos = registros_antes - registros_depois
    pct_removido = (removidos / registros_antes) * 100

    print(f"\n✅ Remoção concluída:")
    print(f"   Antes:    {registros_antes:>7,} linhas")
    print(f"   Removido: {removidos:>7,} linhas ({pct_removido:.1f}%)")
    print(f"   Depois:   {registros_depois:>7,} linhas")

    # Validação: verificar se não removemos demais
    if pct_removido > 50:
        print(f"\n⚠️  ALERTA: Mais de 50% das linhas foram removidas!")
        print(f"   Isso pode indicar falso positivo na detecção.")
        print(f"   Recomenda-se revisar os padrões ou validar manualmente.")

        resposta = input("\n   Deseja MANTER a remoção? (S/N): ").strip().upper()
        if resposta != 'S':
            print(f"\n   ℹ️  Remoção CANCELADA - mantendo dados originais")
            df_sem_totais = df.copy()

    # Atualizar DataFrame principal
    df = df_sem_totais

else:
    print(f"\n✅ Nenhuma linha de totalização detectada - dados já estão limpos!")

# ───────────────────────────────────────────────────────────────────
# ETAPA 3: VALIDAÇÃO FINAL
# ───────────────────────────────────────────────────────────────────

print("\n" + "─" * 80)
print("✅ ETAPA 3: VALIDAÇÃO FINAL")
print("─" * 80)

print(f"\n📊 Dataset final:")
print(f"   Registros: {len(df):,}")
print(f"   Colunas: {len(df.columns)}")

print(f"\n👁️  Preview (primeiras 3 linhas):")
print("─" * 80)
print(df.head(3).to_string())
print("─" * 80)

print("\n" + "="*70)
print("✅ REMOÇÃO DE TOTALIZAÇÕES CONCLUÍDA")
print("="*70)

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# DETECÇÃO AUTOMÁTICA DE CAMPOS (COM DICIONÁRIO PERSISTENTE) -> pedir para detectar nomes de campos unnamed com base no conteudo dos registros, exemplo unnamed com siglas , unnamed com codigos de material, unnamed com siglas, etc e tb remover linhas de totalizacoes, resultados, totais, parciais, em qualquer nivel hierarquivco de agregacao ou granularidade, pois costumam aumentar valores em colunas que tem registros linha a linha (apenas para arquivos tipo bw com hierarquias nao repetidas conforme acima)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🔍 DETECÇÃO AUTOMÁTICA DE CAMPOS")
print("="*70)

# ═══════════════════════════════════════════════════════════════════
# CLASSE DETECTOR DE CAMPOS
# ═══════════════════════════════════════════════════════════════════

class DetectorCampos:
    """Detecta e mapeia campos automaticamente."""

    def __init__(self, df, dicionario_persistente):
        self.df = df
        self.dicionario = dicionario_persistente
        self.mapeamento = {}
        self.relatorio = {
            'total_campos': len(df.columns),
            'mapeados_dicionario': 0,
            'mapeados_heuristica': 0,
            'desconhecidos': 0,
            'ambiguos': 0,
            'detalhes': []
        }

    def detectar_todos(self):
        """Detecta todos os campos do DataFrame."""
        print(f"\n📊 Analisando {len(self.df.columns)} campos...")

        for col in self.df.columns:
            print(f"\n   🔍 Analisando: '{col}'")

            # Extrair amostra de valores
            valores_amostra = self.df[col].dropna().head(100).astype(str).tolist()

            # Tentar detecção
            resultado = self._detectar_campo(col, valores_amostra)

            # Armazenar
            self.mapeamento[col] = resultado

            # Atualizar relatório
            if resultado['metodo'] == 'DICIONARIO':
                self.relatorio['mapeados_dicionario'] += 1
                emoji = "✅"
            elif resultado['metodo'] == 'HEURISTICA':
                self.relatorio['mapeados_heuristica'] += 1
                emoji = "🤖"
            elif resultado['campo_detectado'] == 'DESCONHECIDO':
                self.relatorio['desconhecidos'] += 1
                emoji = "❓"
            else:
                emoji = "⚠️"

            if resultado.get('ambiguidade', False):
                self.relatorio['ambiguos'] += 1
                emoji += "⚠️"

            # Exibir resultado
            print(f"      {emoji} {resultado['campo_detectado']}")
            print(f"      Confiança: {resultado['confianca']:.0%} | Método: {resultado['metodo']}")

            # Adicionar ao relatório
            self.relatorio['detalhes'].append({
                'coluna_original': col,
                'campo_detectado': resultado['campo_detectado'],
                'confianca': resultado['confianca'],
                'metodo': resultado['metodo'],
                'ambiguidade': resultado.get('ambiguidade', False)
            })

        return self.mapeamento, self.relatorio

    def _detectar_campo(self, nome_coluna, valores):
        """Detecta um único campo."""

        # ESTRATÉGIA 1: Buscar no dicionário persistente (MÁXIMA PRIORIDADE)
        if self.dicionario and 'arquivos' in self.dicionario:
            for arquivo_info in self.dicionario.get('arquivos', {}).values():
                if 'campos_mapeados' in arquivo_info:
                    for campo_orig, campo_info in arquivo_info['campos_mapeados'].items():
                        # Match exato no nome
                        if nome_coluna.lower() == campo_orig.lower():
                            return {
                                'campo_detectado': campo_info.get('nome_padrao', campo_orig),
                                'confianca': 1.0,
                                'metodo': 'DICIONARIO',
                                'ambiguidade': False
                            }

                        # Match parcial
                        if nome_coluna.lower() in campo_orig.lower() or campo_orig.lower() in nome_coluna.lower():
                            return {
                                'campo_detectado': campo_info.get('nome_padrao', campo_orig),
                                'confianca': 0.85,
                                'metodo': 'DICIONARIO',
                                'ambiguidade': False
                            }

        # ESTRATÉGIA 2: Heurísticas (se não achou no dicionário)
        resultado_heuristica = self._heuristica_simples(nome_coluna, valores)

        if resultado_heuristica:
            return resultado_heuristica

        # ESTRATÉGIA 3: Desconhecido
        return {
            'campo_detectado': 'DESCONHECIDO',
            'confianca': 0.0,
            'metodo': 'NENHUM',
            'ambiguidade': False
        }

    def _heuristica_simples(self, nome, valores):
        """Heurísticas avançadas por padrões de conteúdo."""

        if not valores:
            return None

        # Remover valores vazios
        valores = [v for v in valores if v and str(v).strip().lower() not in ['nan', 'none', '']]

        if not valores:
            return None

        # ═══════════════════════════════════════════════════════════
        # HEURÍSTICAS PARA CAMPOS UNNAMED (PRIORIDADE MÁXIMA)
        # ═══════════════════════════════════════════════════════════

        if 'unnamed' in nome.lower():
            print(f"      🔍 Analisando conteúdo de '{nome}'...")

            # Amostra de valores para análise
            amostra = valores[:20]

            # U1: Códigos de Centro (4-5 dígitos numéricos)
            if all(str(v).isdigit() and 3 <= len(str(v)) <= 5 for v in amostra):
                return {
                    'campo_detectado': 'Código Centro',
                    'confianca': 0.90,
                    'metodo': 'HEURISTICA_UNNAMED',
                    'ambiguidade': False,
                    'justificativa': 'Códigos numéricos de 3-5 dígitos'
                }

            # U2: Siglas de Base (2-6 letras maiúsculas)
            if all(str(v).isalpha() and str(v).isupper() and 2 <= len(str(v)) <= 6 for v in amostra):
                return {
                    'campo_detectado': 'Sigla Base',
                    'confianca': 0.90,
                    'metodo': 'HEURISTICA_UNNAMED',
                    'ambiguidade': False,
                    'justificativa': 'Siglas em letras maiúsculas'
                }

            # U3: Códigos de Produto (formato: NN.NNN.NNN)
            padrao_produto = r'^\d{2}\.\d{3}\.\d{3}$'
            if all(re.match(padrao_produto, str(v)) for v in amostra):
                return {
                    'campo_detectado': 'Código Produto',
                    'confianca': 0.95,
                    'metodo': 'HEURISTICA_UNNAMED',
                    'ambiguidade': False,
                    'justificativa': 'Formato de código de produto (XX.XXX.XXX)'
                }

            # U4: Códigos Alfanuméricos (possível código de emissor)
            if all(bool(re.search(r'\d', str(v))) and len(str(v)) >= 4 for v in amostra):
                # Verificar se tem letras também
                tem_letras = any(c.isalpha() for v in amostra for c in str(v))
                if tem_letras:
                    return {
                        'campo_detectado': 'Código Emissor',
                        'confianca': 0.80,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': False,
                        'justificativa': 'Códigos alfanuméricos'
                    }
                else:
                    return {
                        'campo_detectado': 'Código Identificador',
                        'confianca': 0.75,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': False,
                        'justificativa': 'Códigos numéricos longos'
                    }

            # U5: Nomes/Descrições (strings longas com espaços)
            tem_espacos = any(' ' in str(v) for v in amostra)
            tamanho_medio = sum(len(str(v)) for v in amostra) / len(amostra)

            if tem_espacos and tamanho_medio > 10:
                # Verificar se é nome de empresa
                palavras_empresa = ['ltda', 'sa', 's.a', 's/a', 'cia', 'company', 'corporation', 'industria', 'comercio']
                eh_empresa = any(any(palavra in str(v).lower() for palavra in palavras_empresa) for v in amostra[:5])

                if eh_empresa:
                    return {
                        'campo_detectado': 'Razão Social',
                        'confianca': 0.85,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': False,
                        'justificativa': 'Nomes de empresas detectados'
                    }
                else:
                    return {
                        'campo_detectado': 'Descrição',
                        'confianca': 0.75,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': False,
                        'justificativa': 'Textos descritivos longos'
                    }

            # U6: Códigos curtos (2-3 caracteres)
            if all(2 <= len(str(v)) <= 3 for v in amostra):
                if all(str(v).isalpha() for v in amostra):
                    return {
                        'campo_detectado': 'Código Curto (Categoria)',
                        'confianca': 0.70,
                        'metodo': 'HEURISTICA_UNNAMED',
                        'ambiguidade': True,
                        'justificativa': 'Códigos alfabéticos curtos'
                    }

            # U7: Estados (2 letras maiúsculas) - comum em BW Brasil
            estados_br = ['AC','AL','AP','AM','BA','CE','DF','ES','GO','MA','MT','MS','MG',
                         'PA','PB','PR','PE','PI','RJ','RN','RS','RO','RR','SC','SP','SE','TO']
            if all(str(v).upper() in estados_br for v in amostra[:10]):
                return {
                    'campo_detectado': 'Estado (UF)',
                    'confianca': 0.95,
                    'metodo': 'HEURISTICA_UNNAMED',
                    'ambiguidade': False,
                    'justificativa': 'Siglas de estados brasileiros'
                }

            # Se chegou aqui e é Unnamed, marcar como desconhecido mas com contexto
            print(f"      ⚠️  Padrão não reconhecido em '{nome}'")
            print(f"      Exemplos: {amostra[:3]}")

        # ═══════════════════════════════════════════════════════════
        # HEURÍSTICAS GERAIS (PARA CAMPOS NÃO-UNNAMED)
        # ═══════════════════════════════════════════════════════════

        # H1: Data no formato YYYY-MM-DD ou DD/MM/YYYY
        padrao_data1 = r'^\d{4}-\d{2}-\d{2}$'
        padrao_data2 = r'^\d{2}/\d{2}/\d{4}$'
        padrao_data3 = r'^\d{1,2}\.\d{4}$'  # M.YYYY ou MM.YYYY

        if all(re.match(padrao_data1, str(v)) or re.match(padrao_data2, str(v)) for v in valores[:10]):
            return {
                'campo_detectado': 'Data',
                'confianca': 0.95,
                'metodo': 'HEURISTICA',
                'ambiguidade': False
            }

        # Formato "M.YYYY" comum em exports BW
        if all(re.match(padrao_data3, str(v)) for v in valores[:10]):
            return {
                'campo_detectado': 'Mês de Calendário',
                'confianca': 0.90,
                'metodo': 'HEURISTICA',
                'ambiguidade': False
            }

        # H2: Números de 4 dígitos (provável código de centro)
        if all(re.match(r'^\d{4}$', str(v)) for v in valores[:10]):
            if 'centro' in nome.lower() or 'cod' in nome.lower() or 'base' in nome.lower():
                return {
                    'campo_detectado': 'Código Centro',
                    'confianca': 0.85,
                    'metodo': 'HEURISTICA',
                    'ambiguidade': False
                }

        # H3: Siglas de 5 letras (formato clássico de bases BR)
        if all(len(str(v)) == 5 and str(v).isalpha() and str(v).isupper() for v in valores[:10]):
            return {
                'campo_detectado': 'Sigla Base',
                'confianca': 0.85,
                'metodo': 'HEURISTICA',
                'ambiguidade': False
            }

        # H4: Percentuais
        if 'porcent' in nome.lower() or '%' in nome or 'perc' in nome.lower():
            return {
                'campo_detectado': 'Percentual',
                'confianca': 0.80,
                'metodo': 'HEURISTICA',
                'ambiguidade': False
            }

        # H5: Valores numéricos grandes (volume/estoque)
        try:
            valores_num = []
            for v in valores[:10]:
                v_str = str(v).replace(',', '.').replace(' ', '')
                valores_num.append(float(v_str))

            if all(v > 1000 for v in valores_num):
                # Identificar tipo por nome ou contexto
                if 'volume' in nome.lower() or 'expedicao' in nome.lower() or 'expedição' in nome.lower():
                    return {
                        'campo_detectado': 'Volume Expedição',
                        'confianca': 0.85,
                        'metodo': 'HEURISTICA',
                        'ambiguidade': False
                    }
                elif 'estoque' in nome.lower() or 'saldo' in nome.lower():
                    return {
                        'campo_detectado': 'Estoque',
                        'confianca': 0.85,
                        'metodo': 'HEURISTICA',
                        'ambiguidade': False
                    }
                elif 'consigna' in nome.lower():
                    return {
                        'campo_detectado': 'Estoque Consignado',
                        'confianca': 0.90,
                        'metodo': 'HEURISTICA',
                        'ambiguidade': False
                    }
                else:
                    return {
                        'campo_detectado': 'Quantidade',
                        'confianca': 0.70,
                        'metodo': 'HEURISTICA',
                        'ambiguidade': True
                    }
        except:
            pass

        # H6: Códigos de Produto (por nome + formato)
        if 'produto' in nome.lower() or 'material' in nome.lower():
            # Verificar se tem formato de código
            if all(any(c.isdigit() for c in str(v)) for v in valores[:10]):
                return {
                    'campo_detectado': 'Código Produto',
                    'confianca': 0.80,
                    'metodo': 'HEURISTICA',
                    'ambiguidade': False
                }

        # H7: Nomes de Produtos/Materiais (por nome + conteúdo textual)
        if 'produto' in nome.lower() or 'material' in nome.lower() or 'descri' in nome.lower():
            tem_texto = any(len(str(v)) > 10 and any(c.isalpha() for c in str(v)) for v in valores[:5])
            if tem_texto:
                return {
                    'campo_detectado': 'Descrição Produto',
                    'confianca': 0.80,
                    'metodo': 'HEURISTICA',
                    'ambiguidade': False
                }

        return None

# ═══════════════════════════════════════════════════════════════════
# EXECUTAR DETECÇÃO
# ═══════════════════════════════════════════════════════════════════

detector = DetectorCampos(df, DICIONARIO_PERSISTENTE)
mapeamento_campos, relatorio_deteccao = detector.detectar_todos()

# ═══════════════════════════════════════════════════════════════════
# RELATÓRIO DE DETECÇÃO
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📋 RELATÓRIO DE DETECÇÃO")
print("="*70)

print(f"\n📊 RESUMO:")
print(f"   Total de campos: {relatorio_deteccao['total_campos']}")
print(f"   ✅ Dicionário: {relatorio_deteccao['mapeados_dicionario']}")
print(f"   🤖 Heurística: {relatorio_deteccao['mapeados_heuristica']}")
print(f"   ❓ Desconhecidos: {relatorio_deteccao['desconhecidos']}")
print(f"   ⚠️  Ambíguos: {relatorio_deteccao['ambiguos']}")

# Taxa de sucesso
taxa_sucesso = ((relatorio_deteccao['mapeados_dicionario'] + relatorio_deteccao['mapeados_heuristica']) /
                relatorio_deteccao['total_campos']) * 100

print(f"\n🎯 Taxa de detecção: {taxa_sucesso:.1f}%")

# Detalhes por confiança
print(f"\n📊 DISTRIBUIÇÃO POR CONFIANÇA:")

alta = sum(1 for d in relatorio_deteccao['detalhes'] if d['confianca'] >= 0.85)
media = sum(1 for d in relatorio_deteccao['detalhes'] if 0.70 <= d['confianca'] < 0.85)
baixa = sum(1 for d in relatorio_deteccao['detalhes'] if 0 < d['confianca'] < 0.70)
zero = sum(1 for d in relatorio_deteccao['detalhes'] if d['confianca'] == 0)

print(f"   🟢 Alta (≥85%): {alta}")
print(f"   🟡 Média (70-85%): {media}")
print(f"   🟠 Baixa (<70%): {baixa}")
print(f"   ⚫ Desconhecidos: {zero}")

# Lista de desconhecidos
desconhecidos = [d for d in relatorio_deteccao['detalhes'] if d['campo_detectado'] == 'DESCONHECIDO']

if desconhecidos:
    print(f"\n❓ CAMPOS DESCONHECIDOS:")
    for item in desconhecidos:
        print(f"   • {item['coluna_original']}")

# ═══════════════════════════════════════════════════════════════════
# APLICAR MAPEAMENTO (CRIAR COLUNAS PADRONIZADAS)
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("🔄 APLICANDO MAPEAMENTO")
print("─"*70)

# Criar dicionário de rename (apenas campos com confiança ≥70%)
rename_dict = {}
for col_orig, info in mapeamento_campos.items():
    if info['confianca'] >= 0.70 and info['campo_detectado'] != 'DESCONHECIDO':
        # Se nome diferente, mapear
        if col_orig != info['campo_detectado']:
            rename_dict[col_orig] = info['campo_detectado']

print(f"\n📝 Mapeamentos a aplicar: {len(rename_dict)}")

if rename_dict:
    # Exibir primeiros 10
    print(f"\n   Primeiros 10 mapeamentos:")
    for i, (orig, novo) in enumerate(list(rename_dict.items())[:10], 1):
        print(f"   {i:2d}. '{orig}' → '{novo}'")

    if len(rename_dict) > 10:
        print(f"   ... e mais {len(rename_dict) - 10}")

    # Aplicar rename
    df_mapeado = df.rename(columns=rename_dict)

    print(f"\n✅ Mapeamento aplicado!")
else:
    df_mapeado = df.copy()
    print(f"\n   ℹ️  Nenhum mapeamento necessário (nomes já padronizados)")

# ═══════════════════════════════════════════════════════════════════
# SALVAR MAPEAMENTO NO DICIONÁRIO PERSISTENTE
# ═══════════════════════════════════════════════════════════════════

print("\n" + "─"*70)
print("💾 ATUALIZANDO DICIONÁRIO PERSISTENTE")
print("─"*70)

# Identificar fonte
nome_fonte = f"CSV_{arquivo_selecionado.stem}"

# Criar entrada no dicionário
if 'arquivos' not in DICIONARIO_PERSISTENTE:
    DICIONARIO_PERSISTENTE['arquivos'] = {}

DICIONARIO_PERSISTENTE['arquivos'][nome_fonte] = {
    'arquivo_origem': arquivo_selecionado.name,
    'data_processamento': datetime.now().isoformat(),
    'total_campos': len(df.columns),
    'campos_mapeados': {}
}

# Adicionar campos com confiança ≥ 85%
for col_orig, info in mapeamento_campos.items():
    if info['confianca'] >= 0.85:
        DICIONARIO_PERSISTENTE['arquivos'][nome_fonte]['campos_mapeados'][col_orig] = {
            'nome_padrao': info['campo_detectado'],
            'confianca': info['confianca'],
            'metodo': info['metodo']
        }

# Salvar dicionário
dict_path = fm.pastas['logs'] / 'DICT_Dicionario_Persistente.json'
with open(dict_path, 'w', encoding='utf-8') as f:
    json.dump(DICIONARIO_PERSISTENTE, f, indent=2, ensure_ascii=False)

print(f"✅ Dicionário atualizado: {dict_path.name}")
print(f"   Novos campos: {len(DICIONARIO_PERSISTENTE['arquivos'][nome_fonte]['campos_mapeados'])}")

# ═══════════════════════════════════════════════════════════════════
# PREVIEW DOS DADOS MAPEADOS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("👀 PREVIEW DOS DADOS MAPEADOS")
print("="*70)

print(f"\n📊 Shape: {df_mapeado.shape[0]:,} registros × {df_mapeado.shape[1]} colunas")

print(f"\n📋 Primeiras 10 colunas:")
for i, col in enumerate(df_mapeado.columns[:10], 1):
    original = [k for k, v in rename_dict.items() if v == col]
    if original:
        print(f"   {i:2d}. {col} (era: {original[0]})")
    else:
        print(f"   {i:2d}. {col}")

print(f"\n📈 Primeiras 3 linhas:")
print(df_mapeado.head(3).to_string())

# Substituir df original
df = df_mapeado

print("\n" + "="*70)
print("✅ DETECÇÃO DE CAMPOS CONCLUÍDA")
print("="*70)

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# GUI VISUAL PARA CONFIRMAR TIPOS - INTERFACE INTUITIVA
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("🎯 CONFIRMAÇÃO VISUAL DE TIPOS")
print("="*70)

if not requer_confirmacao:
    print("\n✅ Nenhuma confirmação necessária - todas com confiança ≥ 90%")
else:
    print(f"\n📋 {len(requer_confirmacao)} campos requerem confirmação")
    print(f"   Abrindo interface visual...")

    def confirmar_tipos_visual(campos_confirmar, tipos_detectados, df):
        """
        GUI VISUAL com lista numerada de tipos
        """
        # Preparar lista de tipos disponíveis
        tipos_dict = dicionario.dados['campos_conhecidos']
        tipos_lista = []

        for i, (nome_tipo, info) in enumerate(sorted(tipos_dict.items()), 1):
            tipos_lista.append({
                'numero': i,
                'nome': nome_tipo,
                'descricao': info.get('descricao', ''),
                'exemplos': info.get('exemplos', [])
            })

        # Adicionar opções especiais
        tipos_lista.append({
            'numero': 0,
            'nome': 'DESCONHECIDO',
            'descricao': 'Tipo não identificado',
            'exemplos': []
        })

        confirmacoes = {}
        idx_atual = [0]

        def processar_proximo():
            if idx_atual[0] >= len(campos_confirmar):
                return

            col = campos_confirmar[idx_atual[0]]
            info_campo = tipos_detectados[col]
            valores_exemplo = df[col].dropna().unique()[:5].tolist()

            # Criar janela
            root = tk.Tk()
            root.title(f"DETECTOR - Confirmação ({idx_atual[0]+1}/{len(campos_confirmar)})")
            root.geometry("900x650")

            x = (root.winfo_screenwidth() // 2) - 450
            y = (root.winfo_screenheight() // 2) - 325
            root.geometry(f"+{x}+{y}")

            frame_principal = tk.Frame(root, bg='white')
            frame_principal.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

            # ═══════════════════════════════════════════════════════════
            # TOPO: Progresso
            # ═══════════════════════════════════════════════════════════
            frame_topo = tk.Frame(frame_principal, bg='#E3F2FD', relief=tk.RAISED, borderwidth=2)
            frame_topo.pack(fill=tk.X, pady=(0, 15))

            tk.Label(
                frame_topo,
                text=f"🎯 Campo {idx_atual[0]+1} de {len(campos_confirmar)}",
                font=('Arial', 12, 'bold'),
                bg='#E3F2FD',
                fg='#1565C0'
            ).pack(pady=8)

            # ═══════════════════════════════════════════════════════════
            # ESQUERDA: Info do campo
            # ═══════════════════════════════════════════════════════════
            frame_conteudo = tk.Frame(frame_principal, bg='white')
            frame_conteudo.pack(fill=tk.BOTH, expand=True)

            # Coluna esquerda
            frame_esquerda = tk.Frame(frame_conteudo, bg='white', width=400)
            frame_esquerda.pack(side=tk.LEFT, fill=tk.BOTH, expand=False, padx=(0, 10))

            tk.Label(
                frame_esquerda,
                text="📋 CAMPO DO ARQUIVO",
                font=('Arial', 11, 'bold'),
                bg='white',
                anchor='w'
            ).pack(fill=tk.X, pady=(0, 10))

            # Box campo
            frame_campo = tk.Frame(frame_esquerda, bg='#FFF9C4', relief=tk.SUNKEN, borderwidth=2)
            frame_campo.pack(fill=tk.X, pady=(0, 10))

            tk.Label(
                frame_campo,
                text=f"🔖 {col}",
                font=('Arial', 10, 'bold'),
                bg='#FFF9C4',
                anchor='w',
                wraplength=380
            ).pack(fill=tk.X, padx=10, pady=(8, 5))

            tk.Label(
                frame_campo,
                text=f"🤖 Detectado: {info_campo['campo_detectado']}",
                font=('Arial', 9),
                bg='#FFF9C4',
                fg='#F57F17',
                anchor='w'
            ).pack(fill=tk.X, padx=10, pady=(0, 3))

            tk.Label(
                frame_campo,
                text=f"📊 Confiança: {info_campo['confianca']:.0%}",
                font=('Arial', 9),
                bg='#FFF9C4',
                fg='#F57F17',
                anchor='w'
            ).pack(fill=tk.X, padx=10, pady=(0, 8))

            # Exemplos
            tk.Label(
                frame_esquerda,
                text="📊 EXEMPLOS DE VALORES",
                font=('Arial', 10, 'bold'),
                bg='white',
                anchor='w'
            ).pack(fill=tk.X, pady=(5, 5))

            frame_exemplos = tk.Frame(frame_esquerda, bg='#F5F5F5', relief=tk.SUNKEN, borderwidth=1)
            frame_exemplos.pack(fill=tk.X)

            for i, val in enumerate(valores_exemplo, 1):
                val_str = str(val)[:50]
                tk.Label(
                    frame_exemplos,
                    text=f"{i}. {val_str}",
                    font=('Arial', 9),
                    bg='#F5F5F5',
                    anchor='w'
                ).pack(fill=tk.X, padx=10, pady=2)

            # ═══════════════════════════════════════════════════════════
            # DIREITA: Lista de tipos
            # ═══════════════════════════════════════════════════════════
            frame_direita = tk.Frame(frame_conteudo, bg='white')
            frame_direita.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

            tk.Label(
                frame_direita,
                text="🔢 TIPOS DISPONÍVEIS - Digite o número",
                font=('Arial', 11, 'bold'),
                bg='white',
                anchor='w'
            ).pack(fill=tk.X, pady=(0, 10))

            # Canvas + Scrollbar para lista
            frame_lista_outer = tk.Frame(frame_direita, bg='white')
            frame_lista_outer.pack(fill=tk.BOTH, expand=True)

            canvas = tk.Canvas(frame_lista_outer, bg='white', highlightthickness=0)
            scrollbar = tk.Scrollbar(frame_lista_outer, orient="vertical", command=canvas.yview)
            frame_lista = tk.Frame(canvas, bg='white')

            frame_lista.bind(
                "<Configure>",
                lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
            )

            canvas.create_window((0, 0), window=frame_lista, anchor="nw")
            canvas.configure(yscrollcommand=scrollbar.set)

            canvas.pack(side="left", fill="both", expand=True)
            scrollbar.pack(side="right", fill="y")

            # Popular lista
            for tipo_info in tipos_lista:
                num = tipo_info['numero']
                nome = tipo_info['nome']
                desc = tipo_info['descricao']

                # Destacar tipo detectado
                bg_cor = '#E8F5E9' if nome == info_campo['campo_detectado'] else 'white'
                fg_cor = '#2E7D32' if nome == info_campo['campo_detectado'] else 'black'

                frame_item = tk.Frame(frame_lista, bg=bg_cor, relief=tk.GROOVE, borderwidth=1)
                frame_item.pack(fill=tk.X, pady=2, padx=5)

                tk.Label(
                    frame_item,
                    text=f"[{num:2d}]  {nome}",
                    font=('Courier', 9, 'bold'),
                    bg=bg_cor,
                    fg=fg_cor,
                    anchor='w'
                ).pack(fill=tk.X, padx=8, pady=(3, 0))

                if desc:
                    tk.Label(
                        frame_item,
                        text=f"      {desc}",
                        font=('Arial', 8),
                        bg=bg_cor,
                        fg='#666666',
                        anchor='w'
                    ).pack(fill=tk.X, padx=8, pady=(0, 3))

            # ═══════════════════════════════════════════════════════════
            # RODAPÉ: Campo de entrada + botões
            # ═══════════════════════════════════════════════════════════
            frame_rodape = tk.Frame(frame_principal, bg='white')
            frame_rodape.pack(fill=tk.X, pady=(15, 0))

            tk.Frame(frame_rodape, height=2, bg='#CCCCCC').pack(fill=tk.X, pady=(0, 10))

            # Campo de entrada
            frame_entrada = tk.Frame(frame_rodape, bg='white')
            frame_entrada.pack(pady=(0, 10))

            tk.Label(
                frame_entrada,
                text="Digite o número do tipo correto:",
                font=('Arial', 10, 'bold'),
                bg='white'
            ).pack(side=tk.LEFT, padx=(0, 10))

            var_numero = tk.StringVar()
            entry = tk.Entry(
                frame_entrada,
                textvariable=var_numero,
                font=('Arial', 12, 'bold'),
                width=8,
                justify='center'
            )
            entry.pack(side=tk.LEFT)
            entry.focus()

            label_erro = tk.Label(
                frame_entrada,
                text="",
                font=('Arial', 9),
                fg='#FF0000',
                bg='white'
            )
            label_erro.pack(side=tk.LEFT, padx=(10, 0))

            # Botões
            frame_btns = tk.Frame(frame_rodape, bg='white')
            frame_btns.pack()

            def validar_e_confirmar():
                try:
                    num_digitado = int(var_numero.get().strip())

                    # Buscar tipo pelo número
                    tipo_escolhido = None
                    for t in tipos_lista:
                        if t['numero'] == num_digitado:
                            tipo_escolhido = t['nome']
                            break

                    if tipo_escolhido:
                        confirmacoes[col] = tipo_escolhido
                        idx_atual[0] += 1
                        root.destroy()
                        processar_proximo()
                    else:
                        label_erro.config(text=f"❌ Número {num_digitado} inválido!")

                except ValueError:
                    label_erro.config(text="❌ Digite um número!")

            def manter_detectado():
                confirmacoes[col] = info_campo['campo_detectado']
                idx_atual[0] += 1
                root.destroy()
                processar_proximo()

            def pular_todos():
                for campo_restante in campos_confirmar[idx_atual[0]:]:
                    confirmacoes[campo_restante] = tipos_detectados[campo_restante]['campo_detectado']
                root.destroy()

            # Enter = confirmar
            entry.bind('<Return>', lambda e: validar_e_confirmar())

            tk.Button(
                frame_btns,
                text="✅ Confirmar",
                command=validar_e_confirmar,
                width=15,
                height=2,
                bg='#4CAF50',
                fg='white',
                font=('Arial', 10, 'bold'),
                cursor='hand2'
            ).pack(side=tk.LEFT, padx=5)

            tk.Button(
                frame_btns,
                text="➡️  Manter Detectado",
                command=manter_detectado,
                width=18,
                height=2,
                bg='#FF9800',
                fg='white',
                font=('Arial', 10),
                cursor='hand2'
            ).pack(side=tk.LEFT, padx=5)

            tk.Button(
                frame_btns,
                text="⏭️  Pular Todos",
                command=pular_todos,
                width=15,
                height=2,
                bg='#757575',
                fg='white',
                font=('Arial', 10),
                cursor='hand2'
            ).pack(side=tk.LEFT, padx=5)

            root.mainloop()

        # Iniciar processamento
        processar_proximo()
        return confirmacoes

    # Executar GUI
    confirmacoes = confirmar_tipos_visual(requer_confirmacao, tipos_detectados, df_limpo)

    # Aplicar confirmações
    if confirmacoes:
        print(f"\n✅ Confirmações aplicadas:")
        for col, tipo_confirmado in confirmacoes.items():
            if tipo_confirmado != tipos_detectados[col]['campo_detectado']:
                print(f"   ✏️  {col}")
                print(f"       {tipos_detectados[col]['campo_detectado']} → {tipo_confirmado}")
                tipos_detectados[col]['campo_detectado'] = tipo_confirmado
                tipos_detectados[col]['confianca'] = 1.0
                tipos_detectados[col]['metodo'] = 'CONFIRMACAO_USUARIO'

        print(f"\n✅ Total: {len(confirmacoes)} campos confirmados")
    else:
        print(f"\n⏭️  Nenhuma confirmação realizada")

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# VALIDAÇÕES E ESTATÍSTICAS
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("📊 VALIDAÇÕES E ESTATÍSTICAS")
print("="*70)

# 1. Resumo Geral
print("\n📋 RESUMO GERAL:")
print("─" * 80)
print(f"   Registros finais: {len(df_limpo):,}")
print(f"   Colunas finais: {len(df_limpo.columns)}")
print(f"   Memória em uso: {df_limpo.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   Linhas duplicadas: {df_limpo.duplicated().sum():,}")

# 2. Valores Nulos
print("\n🔍 ANÁLISE DE VALORES NULOS:")
print("─" * 80)

nulos_por_col = df_limpo.isnull().sum()
colunas_com_nulos = nulos_por_col[nulos_por_col > 0].sort_values(ascending=False)

if len(colunas_com_nulos) > 0:
    print(f"   ⚠️  {len(colunas_com_nulos)} colunas com valores nulos:")
    for col, qtd in colunas_com_nulos.items():
        pct = (qtd / len(df_limpo)) * 100
        barra = "█" * int(pct / 5)
        print(f"      {col:30s} | {qtd:6,} ({pct:5.1f}%) {barra}")
else:
    print(f"   ✅ Nenhum valor nulo!")

# 3. Distribuição de Tipos Detectados
print("\n🔬 DISTRIBUIÇÃO DE TIPOS DETECTADOS:")
print("─" * 80)

tipos_resumo = {}
for info in tipos_detectados.values():
    tipo = info['campo_detectado']  # ✅ CHAVE CORRETA
    tipos_resumo[tipo] = tipos_resumo.get(tipo, 0) + 1

for tipo, count in sorted(tipos_resumo.items(), key=lambda x: x[1], reverse=True):
    barra = "█" * (count * 2)
    print(f"   • {tipo:25s}: {count:2d} colunas {barra}")

# 4. Campos com Alta/Média/Baixa Confiança
print("\n📈 CONFIANÇA NA DETECÇÃO:")
print("─" * 80)

alta = sum(1 for info in tipos_detectados.values() if info['confianca'] >= 0.90)
media = sum(1 for info in tipos_detectados.values() if 0.70 <= info['confianca'] < 0.90)
baixa = sum(1 for info in tipos_detectados.values() if info['confianca'] < 0.70)

print(f"   ✅ Alta (≥90%):   {alta:2d} colunas")
print(f"   ⚠️  Média (70-90%): {media:2d} colunas")
print(f"   ❓ Baixa (<70%):   {baixa:2d} colunas")

# 5. Campos Ambíguos
print("\n⚠️  CAMPOS AMBÍGUOS:")
print("─" * 80)

ambiguos = {col: info for col, info in tipos_detectados.items() if info.get('ambiguidade', False)}

if ambiguos:
    print(f"   {len(ambiguos)} campos com ambiguidade:")
    for col, info in list(ambiguos.items())[:5]:
        print(f"\n   📌 {col}")
        print(f"      Detectado: {info['campo_detectado']}")
        print(f"      Similar a: {', '.join(info.get('candidatos', []))}")

    if len(ambiguos) > 5:
        print(f"\n   ... e mais {len(ambiguos) - 5} campos")
else:
    print(f"   ✅ Nenhum campo ambíguo!")

# 6. Campos Únicos (potenciais IDs)
print("\n🔑 ANÁLISE DE UNICIDADE:")
print("─" * 80)

for col in df_limpo.columns:
    unicos = df_limpo[col].nunique()
    total = len(df_limpo)
    pct_unico = (unicos / total) * 100

    if pct_unico == 100:
        print(f"   🔑 {col:30s} | 100% único (potencial ID)")
    elif pct_unico >= 95:
        print(f"   ⚠️  {col:30s} | {pct_unico:5.1f}% único")

# 7. Cardinalidade (valores únicos)
print("\n📊 CARDINALIDADE:")
print("─" * 80)

for col in df_limpo.columns[:10]:  # Primeiras 10
    unicos = df_limpo[col].nunique()
    total = len(df_limpo)
    pct = (unicos / total) * 100

    if pct <= 10:
        categoria = "Categoria (baixa)"
    elif pct <= 50:
        categoria = "Mista (média)"
    else:
        categoria = "Contínua (alta)"

    print(f"   {col:30s} | {unicos:4d} únicos ({pct:5.1f}%) - {categoria}")

if len(df_limpo.columns) > 10:
    print(f"\n   ... e mais {len(df_limpo.columns) - 10} colunas")

# 8. Tipos de Dados Pandas
print("\n💾 TIPOS DE DADOS (PANDAS):")
print("─" * 80)

dtype_counts = df_limpo.dtypes.value_counts()
for dtype, count in dtype_counts.items():
    print(f"   • {str(dtype):15s}: {count:2d} colunas")

print("\n" + "="*70)
print("✅ VALIDAÇÕES CONCLUÍDAS")
print("="*70)

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# EXPORTAÇÃO DE RESULTADOS - VERSÃO DEFENSIVA
# ═══════════════════════════════════════════════════════════════════

print("\n" + "="*70)
print("💾 EXPORTAÇÃO DE RESULTADOS")
print("="*70)

nome_base = arquivo_selecionado.stem

# 1️⃣  DADOS LIMPOS
print("\n1️⃣  Dados limpos...")
arquivo_limpo = fm.salvar(df_limpo, f"{nome_base}_Limpo", tipo='xlsx', pasta='processados')
print(f"   ✅ {arquivo_limpo.name}")

# 2️⃣  DICIONÁRIO DE CAMPOS
print("\n2️⃣  Dicionário de campos...")
registros_dict = []

for col in df_limpo.columns:
    tipo_info = tipos_detectados.get(col, {})
    valores_exemplo = df_limpo[col].dropna().unique()[:3].tolist()

    registros_dict.append({
        'Coluna': col,
        'Tipo_Detectado': tipo_info.get('campo_detectado', 'DESCONHECIDO'),
        'Confianca_%': tipo_info.get('confianca', 0.0) * 100,
        'Score_Conteudo_%': tipo_info.get('score_conteudo', 0.0) * 100,
        'Score_Nome_%': tipo_info.get('score_nome', 0.0) * 100,
        'Metodo': tipo_info.get('metodo', 'N/A'),
        'Ambiguidade': 'Sim' if tipo_info.get('ambiguidade') else 'Não',
        'Dtype_Pandas': str(df_limpo[col].dtype),
        'Valores_Unicos': df_limpo[col].nunique(),
        'Nulos_Qtd': df_limpo[col].isna().sum(),
        'Nulos_%': (df_limpo[col].isna().sum() / len(df_limpo)) * 100,
        'Exemplo_1': str(valores_exemplo[0]) if len(valores_exemplo) > 0 else None,
        'Exemplo_2': str(valores_exemplo[1]) if len(valores_exemplo) > 1 else None,
        'Exemplo_3': str(valores_exemplo[2]) if len(valores_exemplo) > 2 else None
    })

df_dict = pd.DataFrame(registros_dict)
arquivo_dict = fm.salvar(df_dict, f"DICT_{nome_base}", tipo='xlsx', pasta='outputs')
print(f"   ✅ {arquivo_dict.name}")

# 3️⃣  LOG DE PROCESSAMENTO (COM VERIFICAÇÃO DEFENSIVA)
print("\n3️⃣  Log de processamento...")

# ✅ VERIFICAÇÃO DEFENSIVA - garantir que variáveis existam
try:
    # Tentar usar as variáveis da Célula 8
    _linha_cab_ini = linha_cabecalho_inicio
    _linha_cab_fim = linha_cabecalho_fim
    _idx_cab_ini = idx_cab_inicio
    _idx_cab_fim = idx_cab_fim
    _col_ini = col_inicio
    _col_fim = col_fim
    _idx_col_ini = idx_col_inicio
    _idx_col_fim = idx_col_fim
    _linha_dados_ini_excel = config.get('linha_dados_inicio', linha_cabecalho_inicio + 1)
    _idx_dados_ini = idx_dados_inicio
except NameError:
    # Se não existirem, usar valores padrão/detectados
    print("   ⚠️  Algumas variáveis não encontradas - usando valores padrão")
    _linha_cab_ini = 1
    _linha_cab_fim = 1
    _idx_cab_ini = 0
    _idx_cab_fim = 0
    _col_ini = 1
    _col_fim = len(df_bruto.columns)
    _idx_col_ini = 0
    _idx_col_fim = len(df_bruto.columns)
    _linha_dados_ini_excel = 2
    _idx_dados_ini = 1

log_processamento = {
    'Arquivo_Original': arquivo_selecionado.name,
    'Caminho_Original': str(arquivo_selecionado),
    'Sheet_Processada': sheet_nome,
    'Metodo_Carga': metodo_carga,

    # Cabeçalho
    'Linha_Cabecalho_Inicio_Excel': _linha_cab_ini,
    'Linha_Cabecalho_Fim_Excel': _linha_cab_fim,
    'Indice_Cabecalho_Inicio_Python': _idx_cab_ini,
    'Indice_Cabecalho_Fim_Python': _idx_cab_fim,

    # Colunas
    'Coluna_Inicio_Excel': _col_ini,
    'Coluna_Fim_Excel': _col_fim,
    'Indice_Coluna_Inicio_Python': _idx_col_ini,
    'Indice_Coluna_Fim_Python': _idx_col_fim,

    # Dados
    'Linha_Dados_Inicio_Excel': _linha_dados_ini_excel,
    'Indice_Dados_Inicio_Python': _idx_dados_ini,

    # Contadores
    'Registros_Bruto': len(df_bruto),
    'Colunas_Bruto': len(df_bruto.columns),
    'Registros_Limpo': len(df_limpo),
    'Colunas_Limpo': len(df_limpo.columns),
    'Operacoes_Limpeza': ', '.join(log_limpeza) if 'log_limpeza' in locals() and log_limpeza else 'Nenhuma',

    # Timestamp
    'Timestamp': fm.timestamp,
    'Data_Processamento': datetime.now().isoformat()
}

df_log = pd.DataFrame([log_processamento])
arquivo_log = fm.salvar(df_log, f"LOG_{nome_base}", tipo='xlsx', pasta='logs')
print(f"   ✅ {arquivo_log.name}")

# 4️⃣  CÓDIGO PYTHON PARA REPRODUÇÃO
print("\n4️⃣  Código de reprodução...")
codigo_reprod = f'''# ═══════════════════════════════════════════════════════════════════
# CÓDIGO DE REPRODUÇÃO - Gerado automaticamente
# Arquivo: {arquivo_selecionado.name}
# Sheet: {sheet_nome}
# Data: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
from pathlib import Path

# ═══════════════════════════════════════════════════════════════════
# CONFIGURAÇÃO
# ═══════════════════════════════════════════════════════════════════

arquivo = Path(r"{arquivo_selecionado}")
sheet = "{sheet_nome}"

# Range de extração (índices Python - começa em 0)
linha_cabecalho_inicio = {_idx_cab_ini}
linha_cabecalho_fim = {_idx_cab_fim}
col_inicio = {_idx_col_ini}
col_fim = {_idx_col_fim}
linha_dados_inicio = {_idx_dados_ini}

# ═══════════════════════════════════════════════════════════════════
# CARREGAMENTO
# ═══════════════════════════════════════════════════════════════════

print(f"📂 Carregando: {{arquivo.name}}")
print(f"📄 Sheet: {{sheet}}")

if linha_cabecalho_inicio == linha_cabecalho_fim:
    # CASO 1: Cabeçalho em 1 linha
    print(f"📋 Cabeçalho: Linha {{linha_cabecalho_inicio + 1}} (Excel)")

    df = pd.read_excel(
        arquivo,
        sheet_name=sheet,
        header=linha_cabecalho_inicio,
        usecols=range(col_inicio, col_fim)
    )

    # Pular linhas entre cabeçalho e dados
    linhas_pular = linha_dados_inicio - linha_cabecalho_inicio - 1
    if linhas_pular > 0:
        print(f"⏭️  Pulando {{linhas_pular}} linhas")
        df = df.iloc[linhas_pular:].copy()
else:
    # CASO 2: Cabeçalho em múltiplas linhas
    print(f"📋 Cabeçalho: Linhas {{linha_cabecalho_inicio + 1}} a {{linha_cabecalho_fim + 1}} (Excel)")

    df_temp = pd.read_excel(
        arquivo,
        sheet_name=sheet,
        header=None,
        usecols=range(col_inicio, col_fim)
    )

    # Combinar linhas do cabeçalho
    cabecalho = df_temp.iloc[linha_cabecalho_inicio:linha_cabecalho_fim+1].values
    cab_final = []

    for col_idx in range(cabecalho.shape[1]):
        partes = [
            str(linha[col_idx]).strip()
            for linha in cabecalho
            if str(linha[col_idx]).strip() not in ['', 'nan', 'None']
        ]
        cab_final.append(' - '.join(partes) if partes else f'Col_{{col_idx}}')

    # Extrair dados
    df = df_temp.iloc[linha_dados_inicio:].copy()
    df.columns = cab_final

# Reset index
df = df.reset_index(drop=True)

print(f"✅ Carregado: {{len(df):,}} registros × {{len(df.columns)}} colunas")

# ═══════════════════════════════════════════════════════════════════
# VALIDAÇÃO
# ═══════════════════════════════════════════════════════════════════

colunas_esperadas = {df_limpo.columns.tolist()}

if df.columns.tolist() == colunas_esperadas:
    print("✅ Estrutura validada - colunas correspondem!")
else:
    print("⚠️  Diferença na estrutura:")

    extras = set(df.columns) - set(colunas_esperadas)
    if extras:
        print(f"   Colunas extras: {{extras}}")

    faltando = set(colunas_esperadas) - set(df.columns)
    if faltando:
        print(f"   Colunas faltando: {{faltando}}")

print(f"\\n📊 Shape: {{df.shape}}")
print(f"💾 Memória: {{df.memory_usage(deep=True).sum() / 1024**2:.2f}} MB")
'''

arquivo_codigo = fm.pastas['codigos_integracao'] / f"REPROD_{nome_base}_{fm.timestamp}.py"
with open(arquivo_codigo, 'w', encoding='utf-8') as f:
    f.write(codigo_reprod)

print(f"   ✅ {arquivo_codigo.name}")

# 5️⃣  ATUALIZAR DICIONÁRIO MASTER
print("\n5️⃣  Atualizando dicionário master...")
dicionario.atualizar_historico({
    'arquivo': arquivo_selecionado.name,
    'sheet': sheet_nome,
    'timestamp': fm.timestamp,
    'colunas': df_limpo.columns.tolist(),
    'registros': len(df_limpo),
    'tipos': {col: info['campo_detectado'] for col, info in tipos_detectados.items()}
})
print(f"   ✅ Histórico atualizado")

# 6️⃣  RESUMO DE ARQUIVOS GERADOS
print("\n" + "="*70)
print("📂 ARQUIVOS GERADOS")
print("="*70)

arquivos_gerados = [
    ('📊 Dados Limpos', arquivo_limpo),
    ('📖 Dicionário de Campos', arquivo_dict),
    ('📋 Log de Processamento', arquivo_log),
    ('🐍 Código de Reprodução', arquivo_codigo)
]

for emoji_desc, path in arquivos_gerados:
    print(f"\n{emoji_desc}")
    print(f"   📁 {path.name}")
    print(f"   📂 {path.parent}")
    print(f"   📏 {path.stat().st_size / 1024:.1f} KB")

print("\n" + "="*70)
print("✅ EXPORTAÇÃO CONCLUÍDA COM SUCESSO!")
print("="*70)

# Variável global para uso posterior
df_resultado = df_limpo.copy()

print(f"\n💡 Dataset disponível em: df_resultado")
print(f"   Shape: {df_resultado.shape}")
print(f"   Memória: {df_resultado.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# RELATÓRIO FINAL - RESUMO VISUAL COMPLETO
# ═══════════════════════════════════════════════════════════════════

print("\n")
print("╔" + "="*68 + "╗")
print("║" + " 📋 RELATÓRIO FINAL - PROCESSAMENTO CONCLUÍDO".center(78) + "║")
print("╚" + "="*68 + "╝")

# ═══════════════════════════════════════════════════════════════════
# INFORMAÇÕES DO ARQUIVO
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 📁 INFORMAÇÕES DO ARQUIVO".center(78) + "┃")
print("┣" + "━"*68 + "┫")

print(f"┃  Nome: {arquivo_selecionado.name:<68}┃")
print(f"┃  Sheet: {sheet_nome:<67}┃")

# ✅ CÓDIGO DEFENSIVO - verificar se variáveis existem
try:
    _linha_cab_ini = linha_cabecalho_inicio
    _idx_cab_ini = idx_cab_inicio
    _linha_dados = config.get('linha_dados_inicio', linha_cabecalho_inicio + 1)
    print(f"┃  Cabeçalho: Linha {_linha_cab_ini} (Excel) / Índice {_idx_cab_ini} (Python){' '*20}┃")
    print(f"┃  Dados: A partir da linha {_linha_dados} (Excel){' '*37}┃")
except NameError:
    print(f"┃  Cabeçalho: [informação não disponível]{' '*36}┃")

print(f"┃  Método: {metodo_carga:<66}┃")
print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# ESTATÍSTICAS DE PROCESSAMENTO
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 📊 ESTATÍSTICAS DE PROCESSAMENTO".center(78) + "┃")
print("┣" + "━"*68 + "┫")

print(f"┃  Registros originais: {len(df_bruto):>6,}{' '*47}┃")
print(f"┃  Registros finais:    {len(df_limpo):>6,}{' '*47}┃")
print(f"┃  Diferença:           {len(df_bruto) - len(df_limpo):>6,} removidos{' '*39}┃")
print("┃" + "─"*68 + "┃")
print(f"┃  Colunas originais:   {len(df_bruto.columns):>6,}{' '*47}┃")
print(f"┃  Colunas finais:      {len(df_limpo.columns):>6,}{' '*47}┃")
print(f"┃  Diferença:           {len(df_bruto.columns) - len(df_limpo.columns):>6,} removidas{' '*38}┃")
print("┃" + "─"*68 + "┃")

memoria_mb = df_limpo.memory_usage(deep=True).sum() / 1024**2
print(f"┃  Memória em uso:      {memoria_mb:>6.2f} MB{' '*43}┃")

duplicatas = df_limpo.duplicated().sum()
print(f"┃  Linhas duplicadas:   {duplicatas:>6,}{' '*47}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# QUALIDADE DOS DADOS
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 🔍 QUALIDADE DOS DADOS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

total_nulos = df_limpo.isnull().sum().sum()
total_celulas = len(df_limpo) * len(df_limpo.columns)
pct_nulos = (total_nulos / total_celulas) * 100

print(f"┃  Total de valores nulos: {total_nulos:>6,} ({pct_nulos:>5.2f}%){' '*35}┃")

colunas_com_nulos = df_limpo.isnull().sum()
colunas_com_nulos = colunas_com_nulos[colunas_com_nulos > 0]

if len(colunas_com_nulos) > 0:
    print(f"┃  Colunas com nulos:      {len(colunas_com_nulos):>6,}{' '*47}┃")
else:
    print(f"┃  ✅ Nenhuma coluna com valores nulos!{' '*39}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# DETECÇÃO DE TIPOS
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 🔬 DETECÇÃO DE TIPOS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

alta = sum(1 for info in tipos_detectados.values() if info['confianca'] >= 0.90)
media = sum(1 for info in tipos_detectados.values() if 0.70 <= info['confianca'] < 0.90)
baixa = sum(1 for info in tipos_detectados.values() if info['confianca'] < 0.70)

print(f"┃  ✅ Alta confiança (≥90%):   {alta:>3} colunas{' '*41}┃")
print(f"┃  ⚠️  Média confiança (70-90%): {media:>3} colunas{' '*41}┃")
print(f"┃  ❓ Baixa confiança (<70%):   {baixa:>3} colunas{' '*41}┃")

print("┃" + "─"*68 + "┃")

# Top 5 tipos detectados
tipos_resumo = {}
for info in tipos_detectados.values():
    tipo = info['campo_detectado']
    tipos_resumo[tipo] = tipos_resumo.get(tipo, 0) + 1

print("┃  Top 5 tipos mais comuns:{' '*53}┃")
for i, (tipo, count) in enumerate(sorted(tipos_resumo.items(), key=lambda x: x[1], reverse=True)[:5], 1):
    print(f"┃     {i}. {tipo:<25} ({count:>2} colunas){' '*24}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# OPERAÇÕES DE LIMPEZA
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 🧹 OPERAÇÕES DE LIMPEZA REALIZADAS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

if 'log_limpeza' in locals() and log_limpeza:
    for i, operacao in enumerate(log_limpeza, 1):
        # Quebrar linhas longas
        if len(operacao) > 70:
            print(f"┃  {i}. {operacao[:67]}...┃")
        else:
            print(f"┃  {i}. {operacao:<73}┃")
else:
    print(f"┃  ✅ Nenhuma limpeza necessária - dados já estavam limpos!{' '*18}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# ARQUIVOS GERADOS
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 📂 ARQUIVOS GERADOS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

arquivos_info = [
    ("Dados Limpos", arquivo_limpo),
    ("Dicionário de Campos", arquivo_dict),
    ("Log de Processamento", arquivo_log),
    ("Código de Reprodução", arquivo_codigo)
]

for descricao, path in arquivos_info:
    tamanho_kb = path.stat().st_size / 1024
    nome_arquivo = path.name

    # Truncar nome se muito longo
    if len(nome_arquivo) > 50:
        nome_arquivo = nome_arquivo[:47] + "..."

    print(f"┃  {descricao}:{' '*(30 - len(descricao))}┃")
    print(f"┃     📄 {nome_arquivo:<68}┃")
    print(f"┃     📏 {tamanho_kb:>6.1f} KB{' '*61}┃")
    print("┃" + "─"*68 + "┃")

# Remover última linha divisória
print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# PRÓXIMOS PASSOS
# ═══════════════════════════════════════════════════════════════════

print("\n┏" + "━"*68 + "┓")
print("┃" + " 💡 PRÓXIMOS PASSOS RECOMENDADOS".center(78) + "┃")
print("┣" + "━"*68 + "┫")

sugestoes = []

# Sugestões baseadas em qualidade
if len(colunas_com_nulos) > 0:
    sugestoes.append("Tratar valores nulos nas colunas identificadas")

if baixa > 0:
    sugestoes.append(f"Revisar {baixa} campos com baixa confiança na detecção")

if duplicatas > 0:
    sugestoes.append("Investigar e remover linhas duplicadas")

# Sugestões padrão
sugestoes.extend([
    "Revisar o dicionário de campos gerado",
    "Validar tipos detectados conforme necessidade",
    "Utilizar código de reprodução para reprocessar"
])

for i, sugestao in enumerate(sugestoes[:6], 1):  # Máximo 6 sugestões
    print(f"┃  {i}. {sugestao:<73}┃")

print("┗" + "━"*68 + "┛")

# ═══════════════════════════════════════════════════════════════════
# RODAPÉ
# ═══════════════════════════════════════════════════════════════════

print("\n╔" + "="*68 + "╗")
print("║" + " ✅ PROCESSAMENTO CONCLUÍDO COM SUCESSO!".center(78) + "║")
print("╠" + "="*68 + "╣")
print("║" + f" Timestamp: {fm.timestamp}".ljust(78) + "║")
print("║" + f" Dataset disponível em: df_resultado".ljust(78) + "║")
print("║" + f" Shape: {df_resultado.shape}".ljust(78) + "║")
print("╚" + "="*68 + "╝")

print("\n")

In [ ]:
# ═══════════════════════════════════════════════════════════════════
# ABERTURA AUTOMÁTICA DA PASTA DESTINO
# ═══════════════════════════════════════════════════════════════════

# Abrir pasta de outputs automaticamente
fm.abrir_pasta('outputs')

print("\n💡 TIP: Se a pasta não abriu, o caminho está exibido no relatório acima!")

In [ ]:
df_resultado